In [1]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 设置中文字体（Mac 系统自带）
plt.rcParams['font.sans-serif'] = ['PingFang SC', 'Arial Unicode MS', 'STHeiti', 'STFangsong']
plt.rcParams['axes.unicode_minus'] = False

### 1. 数据预处理

生成merged_df（为特征工程作准备）和sales data（为预测作准备）

In [2]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta

# =============================================================================
# 1. 预处理三个品种的销售数据（增强聚合）
# =============================================================================
def preprocess_sales_data(file_path, sheet_name):
    """预处理单个品种的销售数据，确保每天只有一条记录"""
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # 处理客户列
    if '客户' in df.columns:
        df['客户'] = df['客户'].str.extract(r'(\d+)$').astype(float)
    df = df[df['单据类型'] == '出库']
    
    # 确保日期是日期类型
    df['日期'] = pd.to_datetime(df['日期'])
    
    # 按天聚合销售数据
    agg_dict = {
        '数量': 'sum',  # 每日总数量
        '价格': 'mean',  # 每日平均价格
    }
    
    # 添加其他需要聚合的列
    if '期货市场参考价' in df.columns:
        agg_dict['期货市场参考价'] = 'mean'
    
    # 按日期和客户分组聚合
    daily_sales = df.groupby(['日期', '客户']).agg(agg_dict).reset_index()
    
    # 计算销售额
    daily_sales['销售额'] = daily_sales['数量'] * daily_sales['价格']
    
    # 重命名列
    rename_dict = {
        '日期': '销售日期',
        '数量': '采购数量',
        '价格': '采购价格'
    }
    daily_sales.rename(columns=rename_dict, inplace=True)
    
    # 保留需要的列
    keep_cols = ['销售日期', '采购数量', '采购价格', '客户', '销售额']
    if '期货市场参考价' in daily_sales.columns:
        keep_cols.append('期货市场参考价')
    
    return daily_sales[keep_cols]

# 输入输出路径
input_file = r"/Users/bytedance/Desktop/特征工程补充/采销明细.xls"
output_dir = r"/Users/bytedance/Desktop/特征工程补充/结果"

# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 处理三个品种
products = ['电解镍', '高碳铬铁', '铝锭']
processed_dfs = {}
big_customers_dict = {}  # 存储每个品种的大客户列表

for product in products:
    df = preprocess_sales_data(input_file, product)
    output_path = os.path.join(output_dir, f"处理后的{product}_销售数据.xlsx")
    df.to_excel(output_path, index=False)
    processed_dfs[product] = df
    
    # 计算每个客户的累计销售额
    customer_sales = df.groupby('客户')['销售额'].sum().reset_index()
    
    # 计算累计销售额的阈值（前20%）
    threshold = customer_sales['销售额'].quantile(0.8)
    
    # 识别大客户
    big_customers = customer_sales[customer_sales['销售额'] >= threshold]['客户'].tolist()
    big_customers_dict[product] = big_customers
    
    print(f"{product} 数据处理完成并保存至: {output_path}")
    print(f"{product} 大客户数量: {len(big_customers)}")

# =============================================================================
# 2. 为所有品种创建时间序列数据（添加大客户询价比例，确保每天一条记录）
# =============================================================================
def create_time_series_data(sales_df, inquiry_file, product, big_customers):
    """为指定品种创建时间序列数据，确保每天只有一条记录"""
    # 加载销售数据
    sales_df = sales_df.copy()
    sales_df.rename(columns={'销售日期': 'date'}, inplace=True)
    sales_df['date'] = pd.to_datetime(sales_df['date']).dt.normalize()
    
    if '期货市场参考价' in sales_df.columns:
        sales_df['期货市场参考价'] = sales_df['期货市场参考价'] / 1000

    # 按天聚合销售数据（确保每天只有一条记录）
    agg_dict = {
        '采购数量': 'sum',
        '采购价格': 'mean',
        '销售额': 'sum'
    }
    if '期货市场参考价' in sales_df.columns:
        agg_dict['期货市场参考价'] = 'mean'
    
    # 按日期聚合，确保每天只有一条记录
    daily_sales = sales_df.groupby('date').agg(agg_dict)
    if '期货市场参考价' in sales_df.columns:
        daily_sales.columns = ['demand', 'avg_price', 'future_ref_price', 'sales']
    else:
        daily_sales.columns = ['demand', 'avg_price', 'sales']
    
    # 加载询价数据
    try:
        inquiry_df = pd.read_excel(inquiry_file, sheet_name=f"{product}报价")
        inquiry_df['date'] = pd.to_datetime(inquiry_df['报价时间']).dt.normalize()
        
        # 添加大客户标记
        inquiry_df['is_big_customer'] = inquiry_df['客户编码'].apply(
            lambda x: 1 if x in big_customers else 0
        )
        
        # 按天聚合询价数据（确保每天只有一条记录）
        daily_inquiry = inquiry_df.groupby('date').agg(
            inquiry_count=('客户编码', 'count'),
            our_quote_avg=('报价', 'mean'),
            inquiry_quantity=('报价数量', 'sum'),
            price_std=('报价', 'std'),
            big_customer_inquiries=('is_big_customer', 'sum'),
            big_customer_quantity=('报价数量', lambda x: x[inquiry_df.loc[x.index, 'is_big_customer'] == 1].sum())
        ).reset_index()
        
        # 计算大客户询价比例
        daily_inquiry['big_customer_ratio'] = daily_inquiry['big_customer_inquiries'] / daily_inquiry['inquiry_count']
        daily_inquiry['big_customer_quantity_ratio'] = daily_inquiry['big_customer_quantity'] / daily_inquiry['inquiry_quantity']
        
        # 添加时间特征用于计算上周和上个月
        daily_inquiry['year'] = daily_inquiry['date'].dt.year
        daily_inquiry['month'] = daily_inquiry['date'].dt.month
        daily_inquiry['week'] = daily_inquiry['date'].dt.isocalendar().week
        
        # 计算上周的数据
        daily_inquiry['last_week'] = daily_inquiry['date'] - timedelta(days=7)
        
        # 计算上个月的数据
        daily_inquiry['last_month'] = daily_inquiry['date'] - pd.DateOffset(months=1)
        
        # 创建上周和上个月的数据副本
        last_week_data = daily_inquiry[['date', 'big_customer_ratio', 'big_customer_quantity_ratio']].copy()
        last_week_data.rename(columns={
            'date': 'last_week_date',
            'big_customer_ratio': 'last_week_big_customer_ratio',
            'big_customer_quantity_ratio': 'last_week_big_customer_quantity_ratio'
        }, inplace=True)
        
        last_month_data = daily_inquiry[['date', 'big_customer_ratio', 'big_customer_quantity_ratio']].copy()
        last_month_data.rename(columns={
            'date': 'last_month_date',
            'big_customer_ratio': 'last_month_big_customer_ratio',
            'big_customer_quantity_ratio': 'last_month_big_customer_quantity_ratio'
        }, inplace=True)
        
        # 合并上周数据
        daily_inquiry = daily_inquiry.merge(
            last_week_data,
            left_on='last_week',
            right_on='last_week_date',
            how='left'
        )
        
        # 合并上个月数据
        daily_inquiry = daily_inquiry.merge(
            last_month_data,
            left_on='last_month',
            right_on='last_month_date',
            how='left'
        )
        
        # 清理多余的列
        drop_cols = ['last_week', 'last_month', 'last_week_date', 'last_month_date']
        daily_inquiry.drop(columns=[col for col in drop_cols if col in daily_inquiry.columns], inplace=True)
        
    except Exception as e:
        print(f"警告: 处理{product}询价数据时出错: {e}")
        print(f"将使用空数据")
        daily_inquiry = pd.DataFrame(columns=['date', 'inquiry_count', 'our_quote_avg', 
                                              'inquiry_quantity', 'price_std', 
                                              'big_customer_ratio', 'big_customer_quantity_ratio',
                                              'last_week_big_customer_ratio', 'last_week_big_customer_quantity_ratio',
                                              'last_month_big_customer_ratio', 'last_month_big_customer_quantity_ratio'])
    
    # 创建完整时间序列面板 (2020-2024)
    all_dates = pd.date_range(start='2020-01-01', end='2024-02-29', freq='D')
    full_panel = pd.DataFrame({'date': all_dates})
    
    # 合并数据
    merged_df = full_panel.merge(daily_sales.reset_index(), on='date', how='left')
    
    if not daily_inquiry.empty:
        merged_df = merged_df.merge(daily_inquiry, on='date', how='left')
    else:
        # 如果没有询价数据，添加空列
        merged_df['inquiry_count'] = None
        merged_df['our_quote_avg'] = None
        merged_df['inquiry_quantity'] = None
        merged_df['price_std'] = None
        merged_df['big_customer_ratio'] = None
        merged_df['big_customer_quantity_ratio'] = None
        merged_df['last_week_big_customer_ratio'] = None
        merged_df['last_week_big_customer_quantity_ratio'] = None
        merged_df['last_month_big_customer_ratio'] = None
        merged_df['last_month_big_customer_quantity_ratio'] = None
    
    # 填充缺失值
    merged_df['demand'].fillna(0, inplace=True)
    merged_df['inquiry_quantity'].fillna(0, inplace=True)
    merged_df['inquiry_count'].fillna(0, inplace=True)
    merged_df['big_customer_ratio'].fillna(0, inplace=True)
    merged_df['big_customer_quantity_ratio'].fillna(0, inplace=True)
    
    # 填充上周和上个月的占比
    ratio_cols = ['last_week_big_customer_ratio', 'last_week_big_customer_quantity_ratio',
                  'last_month_big_customer_ratio', 'last_month_big_customer_quantity_ratio']
    
    for col in ratio_cols:
        if col in merged_df.columns:
            merged_df[col].fillna(0, inplace=True)
    
    # 价格列向前向后填充
    price_cols = ['avg_price']
    if 'min_price' in merged_df.columns:
        price_cols.append('min_price')
    if 'max_price' in merged_df.columns:
        price_cols.append('max_price')
    if 'future_ref_price' in merged_df.columns:
        price_cols.append('future_ref_price')
    
    merged_df[price_cols] = merged_df[price_cols].fillna(method='ffill').fillna(method='bfill')
    
    # 询价列向前向后填充
    quote_cols = ['our_quote_avg', 'price_std']
    for col in quote_cols:
        if col in merged_df.columns:
            merged_df[col] = merged_df[col].fillna(method='ffill').fillna(method='bfill')
  
    return merged_df, daily_sales

# 处理所有品种的时间序列
inquiry_file = r"/Users/bytedance/Desktop/特征工程补充/4个品种6年报价数据.xls"

for product in products:
    print(f"\n开始处理{product}的时间序列数据...")
    df = processed_dfs[product]
    big_customers = big_customers_dict[product]
    merged_df, daily_sales = create_time_series_data(df, inquiry_file, product, big_customers)
    
    # 检查是否每天只有一条记录
    date_counts = merged_df['date'].value_counts()
    if any(date_counts > 1):
        print(f"警告: {product}数据中存在一天多条记录的情况")
        print(f"重复日期数量: {len(date_counts[date_counts > 1])}")
        # 合并重复记录
        merged_df = merged_df.groupby('date').first().reset_index()
        print(f"已合并重复记录，现在每天只有一条记录")
    
    # 保存合并数据
    merged_output = os.path.join(output_dir, f"{product}_merged_df.xlsx")
    merged_df.to_excel(merged_output, index=False)
    print(f"{product}合并数据保存至: {merged_output}")
    
    # 提取2024年后的daily_sales
    if not daily_sales.empty:
        daily_sales_2024 = daily_sales[daily_sales.index >= datetime(2024, 3, 1)]
        if not daily_sales_2024.empty:
            daily_output = os.path.join(output_dir, f"{product}_2024_daily_sales.xlsx")
            daily_sales_2024.to_excel(daily_output)
            print(f"{product}2024年销售数据保存至: {daily_output}")
        else:
            print(f"提示: {product}没有2024年后的销售数据")
    else:
        print(f"警告: {product}没有销售数据")

print("\n所有处理完成!")

电解镍 数据处理完成并保存至: /Users/bytedance/Desktop/特征工程补充/结果/处理后的电解镍_销售数据.xlsx
电解镍 大客户数量: 603
高碳铬铁 数据处理完成并保存至: /Users/bytedance/Desktop/特征工程补充/结果/处理后的高碳铬铁_销售数据.xlsx
高碳铬铁 大客户数量: 622
铝锭 数据处理完成并保存至: /Users/bytedance/Desktop/特征工程补充/结果/处理后的铝锭_销售数据.xlsx
铝锭 大客户数量: 240

开始处理电解镍的时间序列数据...


/var/folders/x4/ln8wxptd18sbrbwgzv2_7wjm0000gn/T/ipykernel_85815/2605631414.py:214: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['demand'].fillna(0, inplace=True)
/var/folders/x4/ln8wxptd18sbrbwgzv2_7wjm0000gn/T/ipykernel_85815/2605631414.py:215: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

电解镍合并数据保存至: /Users/bytedance/Desktop/特征工程补充/结果/电解镍_merged_df.xlsx
电解镍2024年销售数据保存至: /Users/bytedance/Desktop/特征工程补充/结果/电解镍_2024_daily_sales.xlsx

开始处理高碳铬铁的时间序列数据...


/var/folders/x4/ln8wxptd18sbrbwgzv2_7wjm0000gn/T/ipykernel_85815/2605631414.py:214: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['demand'].fillna(0, inplace=True)
/var/folders/x4/ln8wxptd18sbrbwgzv2_7wjm0000gn/T/ipykernel_85815/2605631414.py:215: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

高碳铬铁合并数据保存至: /Users/bytedance/Desktop/特征工程补充/结果/高碳铬铁_merged_df.xlsx
高碳铬铁2024年销售数据保存至: /Users/bytedance/Desktop/特征工程补充/结果/高碳铬铁_2024_daily_sales.xlsx

开始处理铝锭的时间序列数据...


/var/folders/x4/ln8wxptd18sbrbwgzv2_7wjm0000gn/T/ipykernel_85815/2605631414.py:214: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['demand'].fillna(0, inplace=True)
/var/folders/x4/ln8wxptd18sbrbwgzv2_7wjm0000gn/T/ipykernel_85815/2605631414.py:215: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

铝锭合并数据保存至: /Users/bytedance/Desktop/特征工程补充/结果/铝锭_merged_df.xlsx
铝锭2024年销售数据保存至: /Users/bytedance/Desktop/特征工程补充/结果/铝锭_2024_daily_sales.xlsx

所有处理完成!


### 2. 安全的增强特征工程

为每个产品创建大量预测特征

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
import holidays
import warnings
from statsmodels.tsa.seasonal import seasonal_decompose

# 忽略警告
warnings.filterwarnings('ignore')

# 设置中文字体
# plt.rcParams['font.sans-serif'] = ['SimHei']  
# plt.rcParams['axes.unicode_minus'] = False  

# =============================================================================
# 特征工程函数（增强版）- 修改了缺失值处理策略
# =============================================================================
def perform_feature_engineering(merged_df, product_name):
    """为指定品种执行特征工程，使用智能填充策略处理缺失值"""
    print(f"\n开始为{product_name}执行特征工程...")
    
    # 1. 添加基本时间特征
    merged_df['year'] = merged_df['date'].dt.year
    merged_df['month'] = merged_df['date'].dt.month
    merged_df['day_of_month'] = merged_df['date'].dt.day
    merged_df['quarter'] = merged_df['date'].dt.quarter
    merged_df['week_of_year'] = merged_df['date'].dt.isocalendar().week.astype(int)
    
    # 创建星期几的数值变量（0=周一，6=周日）
    merged_df['day_of_week_num'] = merged_df['date'].dt.dayofweek
    
    # 创建星期几的01变量（使用英文变量名）
    weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    for i, day_name in enumerate(weekday_names):
        merged_df[f'is_{day_name}'] = (merged_df['day_of_week_num'] == i).astype(int)
    
    # 确保按日期排序
    merged_df = merged_df.sort_values('date')

    
    # 2. 添加节假日特征
    def add_holiday_features(df):
        """添加节假日特征"""
        cn_holidays = holidays.CountryHoliday('CN')
        
        df['is_holiday'] = df['date'].apply(lambda x: 1 if x in cn_holidays else 0)
        df['holiday_name'] = df['date'].apply(lambda x: cn_holidays.get(x, ''))
        df['is_pre_holiday'] = df['date'].apply(lambda x: 1 if (x + timedelta(days=1)) in cn_holidays else 0)
        df['is_post_holiday'] = df['date'].apply(lambda x: 1 if (x - timedelta(days=1)) in cn_holidays else 0)
        
        return df
    
    merged_df = add_holiday_features(merged_df)


    
    # 添加更多时间特征
    def add_time_features(df):
        """添加更多时间特征"""
        df = df.copy()
        df['date'] = pd.to_datetime(df['date'])
        df['day_of_year'] = df['date'].dt.dayofyear
        df['is_weekend'] = (df['day_of_week_num'] >= 5).astype(int)  # 周六和周日
        df['quarter'] = df['date'].dt.quarter
        df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
        df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
        return df

    # 添加滞后特征（扩展更多滞后天数）
    def add_lag_features(df, lags=[1, 2, 3, 5, 6, 7, 14, 30, 60, 90, 180, 365]):
        """添加滞后特征"""
        df = df.copy()
        for lag in lags:
            df[f'demand_lag{lag}'] = df['demand'].shift(lag)
        return df

    # 添加移动平均特征（扩展更多窗口大小）--已修改
    def add_moving_average_features(df, windows=[2, 3, 5, 6, 7, 14, 30, 60, 90, 180, 365]):
        """添加移动平均特征"""
        df = df.copy()
        for window in windows:
            df[f'demand_ma{window}'] = df['demand'].shift(1).rolling(window, min_periods=1).mean()
        return df

    def add_seasonal_features(df):
        """
        添加季节性分解特征，每行只使用历史数据（不包含本行）
        """
        df = df.copy()
        df['seasonal'] = np.nan
        df['trend'] = np.nan
        df['residual'] = np.nan
        
        # 最小需要的数据量（至少2个周期 + 一些额外数据）
        min_data_points = 21  # 3周的数据
        
        for i in range(len(df)):
            if i < min_data_points:
                # 数据不够，保持为NaN
                continue
                
            try:
                # 只使用当前行之前的数据
                historical_data = df['demand'].iloc[:i]  # 不包括当前行i
                
                # 填充缺失值
                temp_demand = historical_data.fillna(method='ffill').fillna(method='bfill').fillna(0)
                
                # 确保有足够的数据
                if len(temp_demand) >= min_data_points and temp_demand.std() > 0:
                    # 进行季节性分解
                    decomposition = seasonal_decompose(
                        temp_demand, 
                        period=7, 
                        model='additive', 
                        extrapolate_trend='freq'
                    )
                    
                    # 取最后一个值作为当前预测
                    df.loc[df.index[i], 'seasonal'] = decomposition.seasonal.iloc[-1]
                    df.loc[df.index[i], 'trend'] = decomposition.trend.iloc[-1]
                    df.loc[df.index[i], 'residual'] = decomposition.resid.iloc[-1]
                    
            except Exception as e:
                print(f"第{i}行季节性分解失败: {e}")
                continue
        
        return df

    # 应用增强的特征工程
    print("应用增强的特征工程...")
    merged_df = add_time_features(merged_df)
    merged_df = add_lag_features(merged_df)
    merged_df = add_moving_average_features(merged_df)
    merged_df = add_seasonal_features(merged_df)
    

    
    # 3. 添加价格相关特征（扩展更多价格特征）
    # 计算不同时间窗口的平均报价
    price_windows = [7, 14, 30, 60, 90, 180]
    for window in price_windows:
        merged_df[f'{window}d_avg_quote'] = merged_df['our_quote_avg'].rolling(window, min_periods=1).mean().shift(1)
        merged_df[f'{window}d_avg_price'] = merged_df['avg_price'].rolling(window, min_periods=1).mean().shift(1)
    
    # 计算价格差异和比率
    merged_df['price_diff'] = merged_df['30d_avg_quote'] - merged_df['30d_avg_price']
    merged_df['price_ratio'] = merged_df['30d_avg_quote'] / (merged_df['30d_avg_price'] + 1e-5)
    
    # 计算价格波动率（不同时间窗口）
    volatility_windows = [7, 14, 30, 60, 90]
    for window in volatility_windows:
        merged_df[f'price_volatility_{window}d'] = merged_df['avg_price'].rolling(window, min_periods=1).std().shift(1)

    
    # 4. 添加滞后特征（扩展更多特征和滞后天数）
    lag_features = ['demand', 'inquiry_quantity', 'inquiry_count', 'our_quote_avg', 'avg_price']
    lag_periods = [1, 2, 3, 5, 6, 7, 14, 30, 60, 90]
    
    for feature in lag_features:
        for lag in lag_periods:
            merged_df[f'{feature}_lag{lag}'] = merged_df[feature].shift(lag)

    
    # 5. 添加移动平均特征（扩展更多特征和窗口大小）
    window_sizes = [2, 3, 5, 6, 7, 14, 30, 60, 90]
    for feature in ['demand', 'inquiry_count', 'our_quote_avg', 'avg_price']:
        for window in window_sizes:
            merged_df[f'{feature}_ma{window}'] = merged_df[feature].rolling(window, min_periods=1).mean().shift(1)
            
    
    # 6. 添加需求-询价比例特征（扩展更多组合）
    merged_df['demand_inquiry_ratio'] = merged_df['demand_lag30'] / (merged_df['inquiry_quantity_lag30'] + 1)
    merged_df['demand_inquiry_diff'] = merged_df['demand_lag30'] - merged_df['inquiry_quantity_lag30']
    
    # 添加价格-需求比率
    merged_df['price_demand_ratio'] = merged_df['avg_price_lag7'] / (merged_df['demand_lag7'] + 1)
    
    
    # 7. 添加季节性因子
    def add_seasonal_factors_no_leak(df):
        """添加季节性因子，避免数据泄漏"""
        df = df.copy()
        df['seasonal_factor'] = np.nan
        
        # 需要足够的历史数据来计算季节性
        min_months = 13  # 至少需要13个月的数据
        
        for i in range(len(df)):
            if i < min_months * 30:  # 粗略估计，假设每月30天
                continue
                
            # 只使用当前行之前的历史数据
            historical_data = df.iloc[:i].copy()
            
            if len(historical_data) > 0:
                # 计算历史数据的月度平均和总体平均
                monthly_avg_hist = historical_data.groupby('month')['demand'].mean()
                overall_avg_hist = historical_data['demand'].mean()
                
                # 获取当前行的月份
                current_month = df.iloc[i]['month']
                
                # 如果历史数据中有该月份的记录
                if current_month in monthly_avg_hist.index:
                    seasonal_factor = monthly_avg_hist[current_month] / overall_avg_hist
                    df.iloc[i, df.columns.get_loc('seasonal_factor')] = seasonal_factor
        
        return df
    merged_df = add_seasonal_factors_no_leak(merged_df)

    
    # 8. 添加星期因子（使用星期几01变量）
    # 计算每周总需求
    def add_weekday_factors_no_leak(df):
        """添加星期因子，避免数据泄漏"""
        df = df.copy()
        
        # 确保有year_week列
        df['year_week'] = df['date'].dt.strftime('%Y-%U')
        df['weekday_factor'] = np.nan
        
        # 需要足够的历史数据（至少8周）
        min_days = 56
        
        for i in range(min_days, len(df)):
            # 只使用历史数据
            historical_data = df.iloc[:i].copy()
            
            if len(historical_data) > 0:
                # 计算历史数据中每周的需求总和
                weekly_totals = historical_data.groupby('year_week')['demand'].sum()
                
                # 为历史数据计算weekday_demand_ratio
                historical_data['weekly_total'] = historical_data['year_week'].map(weekly_totals)
                historical_data['hist_weekday_demand_ratio'] = historical_data['demand'] / (historical_data['weekly_total'] + 1e-5)
                
                # 计算每个星期几的平均占比（基于历史数据）
                weekday_avg_ratio = {}
                
                for dow_num in range(7):
                    mask = historical_data['day_of_week_num'] == dow_num
                    if mask.any():
                        avg_ratio = historical_data.loc[mask, 'hist_weekday_demand_ratio'].mean()
                        weekday_avg_ratio[dow_num] = avg_ratio
                    else:
                        weekday_avg_ratio[dow_num] = 1.0/7  # 默认值
                
                # 为当前行分配星期因子
                current_dow = df.iloc[i]['day_of_week_num']
                df.iloc[i, df.columns.get_loc('weekday_factor')] = weekday_avg_ratio.get(current_dow, 1.0/7)
        
        # 清理临时列
        df.drop(columns=['year_week'], inplace=True)
        
        return df

    # 使用修改后的函数
    merged_df = add_weekday_factors_no_leak(merged_df)


    # 9. 添加期货市场参考价波动特征
    def add_future_volatility_no_leak(df):
        """添加期货波动特征，避免数据泄漏"""
        df = df.copy()
        df['year_week'] = df['date'].dt.strftime('%Y-%U')
        df['is_future_volatile'] = 0  # 默认值
        
        # 需要足够的历史数据
        min_days = 14  # 至少2周的数据
        
        for i in range(min_days, len(df)):
            current_date = df.iloc[i]['date']
            current_week = df.iloc[i]['year_week']
            
            # 只使用历史数据（当前行之前的数据）
            historical_data = df.iloc[:i]
            
            if len(historical_data) > 0:
                # 方案1：使用上一个完整周的数据
                # 找到上一个完整的周
                prev_weeks = historical_data['year_week'].unique()
                if len(prev_weeks) > 0:
                    # 取最近的完整周
                    latest_complete_week = prev_weeks[-1]
                    week_data = historical_data[historical_data['year_week'] == latest_complete_week]
                    
                    if len(week_data) >= 5:  # 确保有足够天数的数据
                        week_min = week_data['avg_price'].min()
                        week_max = week_data['avg_price'].max()
                        
                        if week_min > 0:
                            future_range = week_max - week_min
                            future_range_pct = future_range / week_min
                            
                            # 判断是否波动超过5%
                            if future_range_pct > 0.05:
                                df.iloc[i, df.columns.get_loc('is_future_volatile')] = 1
        
        return df
    merged_df = add_future_volatility_no_leak(merged_df)
    
    # 10. 添加更多时间特征
    merged_df['day_of_year'] = merged_df['date'].dt.dayofyear
    
    
    # # 13. 添加下个月需求特征（用于相关性分析）
    # # 创建月份列
    # merged_df['year_month'] = merged_df['date'].dt.to_period('M')
    
    # # 计算每月平均需求
    # monthly_demand = merged_df.groupby('year_month')['demand'].mean().reset_index()
    # monthly_demand.rename(columns={'demand': 'next_month_demand'}, inplace=True)
    
    # # 将下个月需求添加到原始数据
    # merged_df['next_month'] = merged_df['year_month'] + 1
    # merged_df = merged_df.merge(monthly_demand, left_on='next_month', right_on='year_month', how='left')
    # merged_df.rename(columns={'next_month_demand': 'next_month_demand'}, inplace=True)
    
    # # 删除临时列（如果存在）
    # temp_cols = ['year_month', 'next_month', 'year_month_x', 'year_month_y']
    # cols_to_drop = [col for col in temp_cols if col in merged_df.columns]
    # if cols_to_drop:
    #     merged_df.drop(columns=cols_to_drop, inplace=True)

    
    # 14. 添加组合特征（价格与需求的交互）
    merged_df['price_demand_interaction'] = merged_df['avg_price_lag7'] * merged_df['demand_lag7']
    merged_df['price_inquiry_interaction'] = merged_df['avg_price_lag7'] * merged_df['inquiry_count_lag7']
    
    # 15. 添加变化率特征（修正版）
    # 计算前一期相对于更前一期的变化率
    merged_df['demand_change_1d'] = merged_df['demand'].pct_change(periods=1).shift(1)
    merged_df['demand_change_7d'] = merged_df['demand'].pct_change(periods=7).shift(1)
    merged_df['price_change_1d'] = merged_df['avg_price'].pct_change(periods=1).shift(1)
    merged_df['price_change_7d'] = merged_df['avg_price'].pct_change(periods=7).shift(1)

    # 16. 添加波动率特征（修正版）
    merged_df['demand_volatility_7d'] = merged_df['demand'].rolling(7).std().shift(1)
    merged_df['demand_volatility_30d'] = merged_df['demand'].rolling(30).std().shift(1)
    
    # 17. 添加季节性趋势特征
    merged_df['seasonal_trend'] = merged_df['seasonal'] * merged_df['trend']
    
    # 18. 添加大客户相关特征（新增）
    # 检查是否存在大客户相关特征
    big_customer_features = [
        'big_customer_ratio', 'big_customer_quantity_ratio',
        'last_week_big_customer_ratio', 'last_week_big_customer_quantity_ratio',
        'last_month_big_customer_ratio', 'last_month_big_customer_quantity_ratio'
    ]
    
    existing_features = [col for col in big_customer_features if col in merged_df.columns]
    
    if existing_features:
        print(f"添加大客户相关特征: {len(existing_features)}个")
        
        # 添加大客户特征的滞后特征
        for feature in existing_features:
            for lag in [1, 7, 14, 30]:
                merged_df[f'{feature}_lag{lag}'] = merged_df[feature].shift(lag)
        
        # 添加大客户特征的移动平均
        for feature in existing_features:
            for window in [7, 14, 30]:
                merged_df[f'{feature}_ma{window}'] = merged_df[feature].rolling(window, min_periods=1).mean().shift(1)
        
        # 添加大客户特征的变化率
        # 修正变化率特征
        for feature in existing_features:
            merged_df[f'{feature}_change_1d'] = merged_df[feature].pct_change(periods=1).shift(1)  # 加shift(1)
            merged_df[f'{feature}_change_7d'] = merged_df[feature].pct_change(periods=7).shift(1)  # 加shift(1)
        
        # 添加大客户特征与需求特征的交互
        for feature in existing_features:
            merged_df[f'{feature}_demand_interaction'] = merged_df[feature] * merged_df['demand_lag7']
            merged_df[f'{feature}_price_interaction'] = merged_df[feature] * merged_df['avg_price_lag7']
        
        # 添加大客户特征与询价特征的交互
        for feature in existing_features:
            merged_df[f'{feature}_inquiry_interaction'] = merged_df[feature] * merged_df['inquiry_count_lag7']
        
        # 添加大客户特征与季节因子的交互
        for feature in existing_features:
            merged_df[f'{feature}_seasonal_interaction'] = merged_df[feature] * merged_df['seasonal_factor']
    else:
        print(f"警告: {product_name}没有大客户相关特征")
    
    # 19. 添加行业特定特征
    # 添加产品季节性指数（根据产品类型）
    if product_name == '电解镍':
        # 电解镍在第二季度需求较高
        merged_df['product_seasonal_index'] = np.where(
            merged_df['quarter'] == 2, 1.2, 
            np.where(merged_df['quarter'] == 4, 0.9, 1.0)
        )
    elif product_name == '高碳铬铁':
        # 高碳铬铁在第一季度需求较高
        merged_df['product_seasonal_index'] = np.where(
            merged_df['quarter'] == 1, 1.3, 
            np.where(merged_df['quarter'] == 3, 0.8, 1.0)
        )
    else:  # 铝锭
        # 铝锭在第三季度需求较高
        merged_df['product_seasonal_index'] = np.where(
            merged_df['quarter'] == 3, 1.1, 
            np.where(merged_df['quarter'] == 1, 0.95, 1.0)
        )
    
    # 20. 添加技术指标特征（修正版）

    # 添加需求MACD指标（修正版）
    merged_df['demand_ema12'] = merged_df['demand'].ewm(span=12, adjust=False).mean().shift(1)
    merged_df['demand_ema26'] = merged_df['demand'].ewm(span=26, adjust=False).mean().shift(1) 
    merged_df['demand_macd'] = merged_df['demand_ema12'] - merged_df['demand_ema26']

    # 添加需求RSI指标（修正版）
    delta = merged_df['demand'].diff().shift(1)  # shift差值
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14, min_periods=1).mean()
    avg_loss = loss.rolling(window=14, min_periods=1).mean()
    rs = avg_gain / (avg_loss + 1e-5)
    merged_df['demand_rsi'] = 100 - (100 / (1 + rs))
    
    # =============================================================================
    # 智能填充缺失值（而不是删除）
    # =============================================================================
    # 简单版本：只用前向填充，不会泄漏
    print("执行无泄漏缺失值填充...")

    numeric_cols = merged_df.select_dtypes(include=[np.number]).columns
    demand_related = [col for col in numeric_cols if 'demand' in col or 'inquiry' in col or 'price' in col]

    for col in demand_related:
        # 只用前向填充，不用后向填充
        merged_df[col] = merged_df[col].fillna(method='ffill')
        
        # 如果还有缺失值（最开始的几行），用0填充
        merged_df[col] = merged_df[col].fillna(0)

    other_numeric = [col for col in numeric_cols if col not in demand_related]
    merged_df[other_numeric] = merged_df[other_numeric].fillna(0)
    
    # 2. 填充非数值型特征
    non_numeric_cols = merged_df.select_dtypes(exclude=[np.number]).columns
    for col in non_numeric_cols:
        if col == 'date':
            continue  # 日期列不应该有缺失值
        merged_df[col] = merged_df[col].fillna('unknown')
    
    # 检查是否还有缺失值
    if merged_df.isnull().any().any():
        missing_cols = merged_df.columns[merged_df.isnull().any()].tolist()
        print(f"警告: {product_name}数据中仍有缺失值列: {missing_cols}")
        # 用0填充剩余的缺失值
        merged_df.fillna(0, inplace=True)
    
    print(f"{product_name}特征工程后数据集形状: {merged_df.shape}")
    
    return merged_df


# =============================================================================
# 特征相关性分析函数（增强版）
# =============================================================================
def analyze_feature_correlations(merged_df, product_name, output_dir):
    """全面分析特征与当前需求及下个月平均需求的相关性"""
    print(f"\n开始全面分析{product_name}的特征相关性...")
    
    # 创建相关性分析输出目录
    corr_dir = os.path.join(output_dir, f"{product_name}_特征相关性")
    os.makedirs(corr_dir, exist_ok=True)
    print(f"特征相关性分析结果将保存在: {corr_dir}")
    
    # 1. 计算特征与当前需求的相关性
    numeric_cols = merged_df.select_dtypes(include=['int64', 'float64']).columns
    exclude_cols = ['date', 'holiday_name']
    numeric_cols = [col for col in numeric_cols if col not in exclude_cols]
    
    # 确保星期几特征被包含
    weekday_features = [f'is_{day}' for day in ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']]
    for feature in weekday_features:
        if feature not in numeric_cols and feature in merged_df.columns:
            numeric_cols.append(feature)
            print(f"添加星期几特征: {feature}")
    
    # 打印所有数值列名以验证
    print(f"数值特征列: {list(numeric_cols)}")
    
    # 计算相关系数
    corr_with_demand = merged_df[numeric_cols].corrwith(merged_df['demand'])
    corr_with_demand = corr_with_demand.sort_values(ascending=False)
    
    # # 计算特征与下个月平均需求的相关性
    # corr_with_next_month = merged_df[numeric_cols].corrwith(merged_df['next_month_demand'])
    # corr_with_next_month = corr_with_next_month.sort_values(ascending=False)
    
    # 创建相关性DataFrame
    corr_df = pd.DataFrame({
        'feature': corr_with_demand.index,
        'correlation_with_current_demand': corr_with_demand.values,
        'abs_correlation_current': np.abs(corr_with_demand.values)
    })
    
    # 添加特征描述
    feature_descriptions = {
        'demand': '当前需求',
        'year': '年份',
        'month': '月份',
        'day_of_month': '月中第几天',
        'is_month_start': '是否是月初（1=是，0=否）',
        'is_month_end': '是否是月末（1=是，0=否）',
        'quarter': '季度',
        'week_of_year': '年中第几周',
        'is_holiday': '是否是节假日（1=是，0=否）',
        'is_pre_holiday': '是否是节假日前一天（1=是，0=否）',
        'is_post_holiday': '是否是节假日后一天（1=是，0=否）',
        'day_of_year': '年中第几天',
        'is_weekend': '是否是周末（1=是，0=否）',
        'seasonal': '季节性分量（类似于arima的分解）',
        'trend': '趋势分量（类似于arima的分解）',
        'residual': '残差分量（类似于arima的分解）',
        '30d_avg_quote': '前30天平均报价',
        '30d_avg_price': '前30天平均价格',
        'price_diff': '报价与价格差异',
        'price_ratio': '报价与价格比率',
        'price_volatility': '价格波动率',
        'demand_inquiry_ratio': '需求与询价比例',
        'demand_inquiry_diff': '需求与询价差异',
        'seasonal_factor': '季节性因子（当季节需求在全年的占比）',
        'weekday_factor': '星期因子（当日需求在全周的占比）',
        'inquiry_quantity': '询价量',
        'inquiry_count': '询价次数',
        'avg_price': '期货市场参考价',
        'is_future_volatile': '当周期货价格波动是否超过10%（1=是，0=否）',
        'price_demand_ratio': '价格与需求比率',
        'price_demand_interaction': '价格与需求交互特征',
        'price_inquiry_interaction': '价格与询价交互特征',
        'demand_change_1d': '需求日变化率',
        'demand_change_7d': '需求周变化率',
        'price_change_1d': '价格日变化率',
        'price_change_7d': '价格周变化率',
        'demand_volatility_7d': '需求7天波动率',
        'demand_volatility_30d': '需求30天波动率',
        'seasonal_trend': '季节性趋势特征',
        'product_seasonal_index': '产品季节性指数（基于产品类型）',
        'demand_ema12': '需求12天指数移动平均',
        'demand_ema26': '需求26天指数移动平均',
        'demand_macd': '需求MACD指标（12天和26天EMA的差值）',
        'demand_rsi': '需求RSI指标（14天相对强度指数）'
    }
    
    # 为星期几01变量添加描述
    weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    for day_name in weekday_names:
        feature_descriptions[f'is_{day_name}'] = f'是否为{day_name}（1=是，0=否）'
    
    # 为滞后特征和移动平均特征添加描述
    for lag in [1, 2, 3, 5, 6, 7, 14, 30, 60, 90, 180, 365]:
        feature_descriptions[f'demand_lag{lag}'] = f'滞后{lag}天需求'
        feature_descriptions[f'demand_ma{lag}'] = f'{lag}天移动平均需求'
    
    for feature in ['inquiry_quantity', 'inquiry_count', 'our_quote_avg', 'avg_price']:
        for lag in [1, 2, 3, 5, 6, 7, 14, 30, 60, 90]:
            feature_descriptions[f'{feature}_lag{lag}'] = f'滞后{lag}天{feature}'
    
    # 为价格特征添加描述
    for window in [7, 14, 30, 60, 90, 180]:
        feature_descriptions[f'{window}d_avg_quote'] = f'前{window}天平均报价'
        feature_descriptions[f'{window}d_avg_price'] = f'前{window}天平均价格'
    
    for window in [7, 14, 30, 60, 90]:
        feature_descriptions[f'price_volatility_{window}d'] = f'{window}天价格波动率'
    
    # 为大客户特征添加描述
    big_customer_features = [
        'big_customer_ratio', 'big_customer_quantity_ratio',
        'last_week_big_customer_ratio', 'last_week_big_customer_quantity_ratio',
        'last_month_big_customer_ratio', 'last_month_big_customer_quantity_ratio'
    ]
    
    big_customer_descriptions = {
        'big_customer_ratio': '大客户询价次数占比',
        'big_customer_quantity_ratio': '大客户询价数量占比',
        'last_week_big_customer_ratio': '上周大客户询价次数占比',
        'last_week_big_customer_quantity_ratio': '上周大客户询价数量占比',
        'last_month_big_customer_ratio': '上个月大客户询价次数占比',
        'last_month_big_customer_quantity_ratio': '上个月大客户询价数量占比'
    }
    
    # 为大客户衍生特征添加描述
    for feature in big_customer_features:
        # 滞后特征
        for lag in [1, 7, 14, 30]:
            feature_descriptions[f'{feature}_lag{lag}'] = f'滞后{lag}天{big_customer_descriptions[feature]}'
        
        # 移动平均特征
        for window in [7, 14, 30]:
            feature_descriptions[f'{feature}_ma{window}'] = f'{window}天移动平均{big_customer_descriptions[feature]}'
        
        # 变化率特征
        feature_descriptions[f'{feature}_change_1d'] = f'{big_customer_descriptions[feature]}日变化率'
        feature_descriptions[f'{feature}_change_7d'] = f'{big_customer_descriptions[feature]}周变化率'
        
        # 交互特征
        feature_descriptions[f'{feature}_demand_interaction'] = f'{big_customer_descriptions[feature]}与需求交互特征'
        feature_descriptions[f'{feature}_price_interaction'] = f'{big_customer_descriptions[feature]}与价格交互特征'
        feature_descriptions[f'{feature}_inquiry_interaction'] = f'{big_customer_descriptions[feature]}与询价交互特征'
        feature_descriptions[f'{feature}_seasonal_interaction'] = f'{big_customer_descriptions[feature]}与季节因子交互特征'
    
    # 添加特征描述到相关性DataFrame
    corr_df['description'] = corr_df['feature'].map(feature_descriptions)
    
    # # 排序（按下个月绝对相关性降序）
    # corr_df = corr_df.sort_values('abs_correlation_next_month', ascending=False)
    
    # 保存相关性结果
    corr_output = os.path.join(corr_dir, f"{product_name}_特征相关性.xlsx")
    corr_df.to_excel(corr_output, index=False)
    print(f"特征相关性结果已保存至: {corr_output}")
    
    # 2. 可视化所有特征的相关性
    # 创建目标变量列表
    targets = ['demand']
    
    # 为每个目标变量创建相关性图表
    for target in targets:
        # 计算与目标变量的相关性
        corr_with_target = merged_df[numeric_cols].corrwith(merged_df[target])
        corr_with_target = corr_with_target.sort_values(ascending=False)
        
        # 创建相关性DataFrame
        target_corr_df = pd.DataFrame({
            'feature': corr_with_target.index,
            f'correlation_with_{target}': corr_with_target.values,
            f'abs_correlation_{target}': np.abs(corr_with_target.values)
        })
        
        # 添加特征描述
        target_corr_df['description'] = target_corr_df['feature'].map(feature_descriptions)
        
        # 排序（按绝对相关性降序）
        target_corr_df = target_corr_df.sort_values(f'abs_correlation_{target}', ascending=False)
        
        # 保存相关性结果
        target_corr_output = os.path.join(corr_dir, f"{product_name}_与{target}相关性.xlsx")
        target_corr_df.to_excel(target_corr_output, index=False)
        print(f"与{target}相关性结果已保存至: {target_corr_output}")
        
        # 可视化前30个特征的相关性
        plt.figure(figsize=(16, 20))
        top_features = target_corr_df.head(30)['feature'].tolist()
        top_corr = target_corr_df.head(30)[f'correlation_with_{target}'].values
        
        # 创建水平条形图
        plt.barh(top_features, top_corr, color='skyblue')
        plt.xlabel('相关系数')
        plt.ylabel('特征')
        plt.title(f'{product_name}特征与{target}相关性 (Top 30)')
        plt.tight_layout()
        plt.savefig(os.path.join(corr_dir, f'{product_name}_与{target}相关性.png'), dpi=300)
        plt.close()
        print(f"与{target}相关性图已保存至: {os.path.join(corr_dir, f'{product_name}_与{target}相关性.png')}")
    
    # 3. 可视化相关性矩阵
    # 选择所有特征
    all_features = numeric_cols + ['demand']
    
    # 计算相关系数矩阵
    corr_matrix = merged_df[all_features].corr()
    
    # 保存完整相关性矩阵
    corr_matrix_output = os.path.join(corr_dir, f"{product_name}_所有特征相关性矩阵.xlsx")
    corr_matrix.to_excel(corr_matrix_output)
    print(f"所有特征相关性矩阵已保存至: {corr_matrix_output}")
    
    # 可视化（仅显示与目标变量相关性较高的特征）
    # 选择与下个月需求相关性最高的30个特征
    top_features = corr_df.head(30)['feature'].tolist()
    if 'demand' not in top_features:
        top_features.append('demand')
    # if 'next_month_demand' not in top_features:
    #     top_features.append('next_month_demand')
    
    # 计算相关系数矩阵
    corr_matrix = merged_df[top_features].corr()
    
    # 可视化
    plt.figure(figsize=(24, 20))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', center=0)
    plt.title(f'{product_name}特征相关性矩阵 (Top 30)')
    plt.tight_layout()
    plt.savefig(os.path.join(corr_dir, f'{product_name}_特征相关性矩阵.png'), dpi=300)
    plt.close()
    print(f"特征相关性矩阵图已保存至: {os.path.join(corr_dir, f'{product_name}_特征相关性矩阵.png')}")
    
    # # 4. 可视化与下个月需求相关性最高的特征
    # top_corr_features = corr_df.head(20)['feature'].tolist()
    
    # # 创建子图
    # fig, axes = plt.subplots(5, 4, figsize=(24, 30))
    # axes = axes.flatten()
    
    # for i, feature in enumerate(top_corr_features):
    #     if feature == 'next_month_demand':
    #         continue
            
    #     # 绘制散点图
    #     sns.scatterplot(data=merged_df, x=feature, y='next_month_demand', ax=axes[i])
    #     axes[i].set_title(f'{feature} vs 下个月需求 (r={corr_with_next_month[feature]:.2f})')
    #     axes[i].set_xlabel(feature_descriptions.get(feature, feature))
    #     axes[i].set_ylabel('下个月平均需求')
    
    # plt.tight_layout()
    # plt.savefig(os.path.join(corr_dir, f'{product_name}_与下个月需求相关性最高的特征.png'), dpi=300)
    # plt.close()
    # print(f"与下个月需求相关性最高的特征图已保存至: {os.path.join(corr_dir, f'{product_name}_与下个月需求相关性最高的特征.png')}")
    
    # 5. 添加星期几特征的可视化
    weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekday_demand = []
    weekday_next_month_demand = []
    
    for day_name in weekday_names:
        col_name = f'is_{day_name}'
        if col_name in merged_df.columns:
            day_demand = merged_df.loc[merged_df[col_name] == 1, 'demand'].mean()
            weekday_demand.append(day_demand)
    
    if weekday_demand:
        # 当日需求
        plt.figure(figsize=(12, 8))
        plt.bar(weekday_names, weekday_demand)
        plt.title(f'{product_name}各星期几的平均需求')
        plt.xlabel('星期几')
        plt.ylabel('平均需求')
        plt.tight_layout()
        plt.savefig(os.path.join(corr_dir, f'{product_name}_星期几平均需求.png'), dpi=300)
        plt.close()
        print(f"星期几平均需求图已保存至: {os.path.join(corr_dir, f'{product_name}_星期几平均需求.png')}")
        
        # # 下月需求
        # plt.figure(figsize=(12, 8))
        # plt.bar(weekday_names, weekday_next_month_demand)
        # plt.title(f'{product_name}各星期几的下月平均需求')
        # plt.xlabel('星期几')
        # plt.ylabel('下月平均需求')
        # plt.tight_layout()
        # plt.savefig(os.path.join(corr_dir, f'{product_name}_星期几下月平均需求.png'), dpi=300)
        # plt.close()
        # print(f"星期几下月平均需求图已保存至: {os.path.join(corr_dir, f'{product_name}_星期几下月平均需求.png')}")
    
    # 6. 添加期货波动特征的可视化
    if 'is_future_volatile' in merged_df.columns:
        # 期货波动状态下的需求分布
        plt.figure(figsize=(12, 8))
        sns.boxplot(data=merged_df, x='is_future_volatile', y='demand')
        plt.title(f'{product_name}期货波动状态下的需求分布')
        plt.xlabel('期货波动是否超过10%')
        plt.ylabel('需求')
        plt.xticks([0, 1], ['否', '是'])
        plt.tight_layout()
        plt.savefig(os.path.join(corr_dir, f'{product_name}_期货波动需求分布.png'), dpi=300)
        plt.close()
        print(f"期货波动需求分布图已保存至: {os.path.join(corr_dir, f'{product_name}_期货波动需求分布.png')}")
        
        # # 期货波动状态与下个月需求的关系
        # plt.figure(figsize=(12, 8))
        # sns.boxplot(data=merged_df, x='is_future_volatile', y='next_month_demand')
        # plt.title(f'{product_name}期货波动状态与下个月需求')
        # plt.xlabel('期货波动是否超过10%')
        # plt.ylabel('下个月平均需求')
        # plt.xticks([0, 1], ['否', '是'])
        # plt.tight_layout()
        # plt.savefig(os.path.join(corr_dir, f'{product_name}_期货波动与下个月需求.png'), dpi=300)
        # plt.close()
        # print(f"期货波动与下个月需求图已保存至: {os.path.join(corr_dir, f'{product_name}_期货波动与下个月需求.png')}")
    
    # 7. 添加滞后特征的可视化
    lag_features = [f'demand_lag{lag}' for lag in [1, 7, 14, 30, 60, 90]]
    for lag_feature in lag_features:
        if lag_feature in merged_df.columns:
            plt.figure(figsize=(10, 6))
            sns.scatterplot(data=merged_df, x=lag_feature, y='demand')
            plt.title(f'{lag_feature} vs 当前需求')
            plt.xlabel(lag_feature)
            plt.ylabel('当前需求')
            plt.tight_layout()
            plt.savefig(os.path.join(corr_dir, f'{product_name}_{lag_feature}_vs_当前需求.png'), dpi=300)
            plt.close()
            
            # plt.figure(figsize=(10, 6))
            # sns.scatterplot(data=merged_df, x=lag_feature, y='next_month_demand')
            # plt.title(f'{lag_feature} vs 下个月需求')
            # plt.xlabel(lag_feature)
            # plt.ylabel('下个月需求')
            # plt.tight_layout()
            # plt.savefig(os.path.join(corr_dir, f'{product_name}_{lag_feature}_vs_下个月需求.png'), dpi=300)
            # plt.close()
    
    # 8. 添加价格-需求交互特征的可视化
    if 'price_demand_interaction' in merged_df.columns:
        plt.figure(figsize=(10, 6))
        sns.scatterplot(data=merged_df, x='price_demand_interaction', y='demand')
        plt.title('价格-需求交互特征 vs 需求')
        plt.xlabel('价格-需求交互特征')
        plt.ylabel('需求')
        plt.tight_layout()
        plt.savefig(os.path.join(corr_dir, f'{product_name}_价格需求交互特征_vs_需求.png'), dpi=300)
        plt.close()
    
    # 9. 添加变化率特征的可视化
    if 'demand_change_7d' in merged_df.columns:
        plt.figure(figsize=(10, 6))
        sns.scatterplot(data=merged_df, x='demand_change_7d', y='demand')
        plt.title('需求7天变化率 vs 需求')
        plt.xlabel('需求7天变化率')
        plt.ylabel('需求')
        plt.tight_layout()
        plt.savefig(os.path.join(corr_dir, f'{product_name}_需求变化率_vs_需求.png'), dpi=300)
        plt.close()
    
    # 10. 添加大客户特征的可视化
    big_customer_features = [
        'big_customer_ratio', 'last_week_big_customer_ratio', 'last_month_big_customer_ratio'
    ]
    
    for feature in big_customer_features:
        if feature in merged_df.columns:
            plt.figure(figsize=(10, 6))
            sns.scatterplot(data=merged_df, x=feature, y='demand')
            plt.title(f'{feature} vs 需求')
            plt.xlabel(feature_descriptions.get(feature, feature))
            plt.ylabel('需求')
            plt.tight_layout()
            plt.savefig(os.path.join(corr_dir, f'{product_name}_{feature}_vs_需求.png'), dpi=300)
            plt.close()
    
    return merged_df



# =============================================================================
# 修复函数
# =============================================================================


def fix_data_types_before_model(merged_df):
    """
    在模型训练前修复数据类型问题
    """
    print("=== 修复数据类型问题 ===")
    
    # 1. 删除所有可能的字符串列（临时列）
    string_cols_to_drop = []
    for col in merged_df.columns:
        if merged_df[col].dtype == 'object' and col not in ['date', 'holiday_name']:
            # 检查是否包含 '2023-27' 这样的值
            if merged_df[col].astype(str).str.contains(r'\d{4}-\d+', na=False).any():
                print(f"发现包含年-周格式的列: {col}")
                string_cols_to_drop.append(col)
            elif col.startswith('year_week'):
                print(f"发现年周临时列: {col}")
                string_cols_to_drop.append(col)
    
    # 删除这些列
    if string_cols_to_drop:
        print(f"删除字符串列: {string_cols_to_drop}")
        merged_df = merged_df.drop(columns=string_cols_to_drop)
    
    # 2. 强制转换所有应该是数值的列
    exclude_cols = ['date', 'holiday_name']
    numeric_cols = [col for col in merged_df.columns if col not in exclude_cols]
    
    for col in numeric_cols:
        # 检查这一列是否有非数值数据
        original_dtype = merged_df[col].dtype
        
        if original_dtype == 'object':
            print(f"发现对象类型列: {col}")
            # 尝试转换为数值
            numeric_series = pd.to_numeric(merged_df[col], errors='coerce')
            
            # 检查有多少值无法转换
            non_convertible = merged_df[col][numeric_series.isna() & merged_df[col].notna()]
            if len(non_convertible) > 0:
                print(f"  无法转换的值: {non_convertible.unique()[:5]}")
                
            # 用数值数据替换
            merged_df[col] = numeric_series
            print(f"  转换为数值类型: {original_dtype} -> {merged_df[col].dtype}")
        
        elif 'int' in str(original_dtype) or 'float' in str(original_dtype):
            # 已经是数值类型，检查是否有异常
            continue
        else:
            print(f"警告: 意外的数据类型 {col}: {original_dtype}")
    
    # 3. 填充转换后产生的 NaN 值
    numeric_cols = merged_df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if merged_df[col].isnull().sum() > 0:
            # 用前向填充，然后用0填充
            merged_df[col] = merged_df[col].fillna(method='ffill').fillna(0)
            print(f"填充 {col} 的缺失值")
    
    # 4. 最终检查
    print("\n=== 最终数据类型检查 ===")
    non_numeric_cols = merged_df.select_dtypes(exclude=[np.number]).columns
    non_numeric_cols = [col for col in non_numeric_cols if col not in ['date', 'holiday_name']]
    
    if len(non_numeric_cols) > 0:
        print(f"仍然存在非数值列: {non_numeric_cols}")
        for col in non_numeric_cols:
            print(f"  {col}: {merged_df[col].dtype}, 样本值: {merged_df[col].head().tolist()}")
    else:
        print("✓ 所有特征列都是数值类型")
    
    print(f"最终数据形状: {merged_df.shape}")
    return merged_df




# =============================================================================
# 主程序
# =============================================================================

# 输入输出路径
output_dir = r"/Users/bytedance/Desktop/特征工程补充/结果"

# 处理三个品种
products = ['电解镍', '高碳铬铁', '铝锭']

for product in products:
    print(f"\n{'='*80}")
    print(f"开始处理 {product} 的特征工程和相关性分析")
    print(f"{'='*80}")
    
    # 创建产品输出目录
    product_output_dir = os.path.join(output_dir, f"{product}预测结果")
    os.makedirs(product_output_dir, exist_ok=True)
    print(f"输出目录: {product_output_dir}")
    
    # 1. 读取之前生成的合并数据
    input_path = os.path.join(output_dir, f"{product}_merged_df.xlsx")
    if not os.path.exists(input_path):
        print(f"警告: 找不到 {product} 的合并数据文件: {input_path}")
        continue
    
    merged_df = pd.read_excel(input_path)
    
    # 检查数据是否为空
    if merged_df.empty:
        print(f"警告: {product} 的合并数据为空，跳过处理")
        continue
    
    # 确保日期列是datetime类型
    merged_df['date'] = pd.to_datetime(merged_df['date'], errors='coerce')
    
    # 2. 执行特征工程
    feature_df = perform_feature_engineering(merged_df, product)

    feature_df = fix_data_types_before_model(feature_df)
    
    # 保存特征工程后的数据
    feature_output = os.path.join(product_output_dir, f"{product}_特征工程数据.xlsx")
    feature_df.to_excel(feature_output, index=False)
    print(f"{product}特征工程数据保存至: {feature_output}")
    
    # 3. 执行特征相关性分析
    analyze_feature_correlations(feature_df, product, product_output_dir)
    
    print(f"\n{product} 的特征工程和相关性分析完成!")

print("\n所有处理完成!")




开始处理 电解镍 的特征工程和相关性分析
输出目录: /Users/bytedance/Desktop/特征工程补充/结果/电解镍预测结果

开始为电解镍执行特征工程...
应用增强的特征工程...
添加大客户相关特征: 6个
执行无泄漏缺失值填充...
电解镍特征工程后数据集形状: (1521, 249)
=== 修复数据类型问题 ===
发现包含年-周格式的列: year_week
删除字符串列: ['year_week']

=== 最终数据类型检查 ===
✓ 所有特征列都是数值类型
最终数据形状: (1521, 248)
电解镍特征工程数据保存至: /Users/bytedance/Desktop/特征工程补充/结果/电解镍预测结果/电解镍_特征工程数据.xlsx

开始全面分析电解镍的特征相关性...
特征相关性分析结果将保存在: /Users/bytedance/Desktop/特征工程补充/结果/电解镍预测结果/电解镍_特征相关性
数值特征列: ['demand', 'avg_price', 'sales', 'inquiry_count', 'our_quote_avg', 'inquiry_quantity', 'price_std', 'big_customer_inquiries', 'big_customer_quantity', 'big_customer_ratio', 'big_customer_quantity_ratio', 'week', 'last_week_big_customer_ratio', 'last_week_big_customer_quantity_ratio', 'last_month_big_customer_ratio', 'last_month_big_customer_quantity_ratio', 'week_of_year', 'is_Monday', 'is_Tuesday', 'is_Wednesday', 'is_Thursday', 'is_Friday', 'is_Saturday', 'is_Sunday', 'is_holiday', 'is_pre_holiday', 'is_post_holiday', 'is_weekend', 'is_month_start', 'is_

### 3. 特征选择

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from quantile_forest import RandomForestQuantileRegressor
from sklearn.base import clone
import warnings

# 忽略警告
warnings.filterwarnings('ignore')

# 设置中文字体
# plt.rcParams['font.sans-serif'] = ['SimHei']  
# plt.rcParams['axes.unicode_minus'] = False  

# =============================================================================
# 辅助函数 - 数据清理
# =============================================================================
def clean_data(df):
    """清理数据：处理无穷大值、过大值和缺失值"""
    print("执行数据清理...")
    
    # 处理无穷大值
    for col in df.select_dtypes(include=[np.number]).columns:
        # 替换无穷大值为NaN
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)
        
        # 检查是否有过大值
        col_min = df[col].min()
        col_max = df[col].max()
        
        if col_max > 1e10 or col_min < -1e10:
            print(f"  警告: {col} 包含过大值 (min={col_min}, max={col_max})")
            
            # 使用中位数替换过大值
            median_val = df[col].median()
            df[col] = df[col].apply(
                lambda x: median_val if x > 1e10 or x < -1e10 else x
            )
    
    # 填充NaN值
    for col in df.columns:
        if df[col].isna().any():
            if df[col].dtype == 'object':
                # 分类列用众数填充
                mode_val = df[col].mode()[0]
                df[col] = df[col].fillna(mode_val)
            else:
                # 数值列用中位数填充
                median_val = df[col].median()
                df[col] = df[col].fillna(median_val)
    
    # 删除有缺失值的行
    original_rows = df.shape[0]
    df = df.dropna()
    dropped_rows = original_rows - df.shape[0]
    print(f"删除了 {dropped_rows} 行缺失数据。")
    
    return df

# =============================================================================
# 特征过滤函数 - 排除不能当天获得的特征
# =============================================================================
def filter_available_features(feature_cols):
    """
    过滤出在实际应用中当天能获得的特征
    排除不能当天获得的基础特征及其衍生特征
    """
    print("开始过滤特征，排除实际应用中当天无法获得的特征...")
    
    # 明确不能当天获得的基础特征
    forbidden_base_features = {
        'demand', 'avg_price', 'sales', 'inquiry_count', 'our_quote_avg', 
        'inquiry_quantity', 'price_std', 'big_customer_inquiries', 
        'big_customer_quantity', 'big_customer_ratio', 'big_customer_quantity_ratio'
    }
    
    available_features = []
    filtered_features = []
    
    for feature in feature_cols:
        # 检查是否是禁用的基础特征
        is_forbidden = False
        
        for forbidden in forbidden_base_features:
            if feature == forbidden:  # 完全匹配禁用特征
                is_forbidden = True
                filtered_features.append(feature)
                break
            # # 检查是否是基于禁用特征的衍生特征（但没有足够的滞后）
            # elif forbidden in feature:
            #     # 如果包含禁用特征名但没有滞后保护，则排除
            #     if not any(lag_word in feature for lag_word in ['lag', 'ma', 'shift', '_change_']):
            #         is_forbidden = True
            #         filtered_features.append(feature)
            #         break
            #     # 如果是lag1相关的特征，也排除（当天可能无法获得）
            #     elif 'lag1' in feature:
            #         is_forbidden = True
            #         filtered_features.append(feature)
            #         break
        
        if not is_forbidden:
            available_features.append(feature)
    
    print(f"原始特征数量: {len(feature_cols)}")
    print(f"可用特征数量: {len(available_features)}")
    print(f"被过滤特征数量: {len(filtered_features)}")
    
    if filtered_features:
        print(f"被过滤的特征包括:")
        for i, feature in enumerate(filtered_features):
            print(f"  {i+1}. {feature}")
    
    print(f"保留的特征类型包括:")
    feature_types = {
        'time': [],
        'lag': [],
        'ma': [],
        'seasonal': [],
        'other': []
    }
    
    for feature in available_features:
        if any(keyword in feature for keyword in ['year', 'month', 'day', 'quarter', 'week', 'is_', 'holiday']):
            feature_types['time'].append(feature)
        elif 'lag' in feature and 'lag1' not in feature:
            feature_types['lag'].append(feature)
        elif 'ma' in feature:
            feature_types['ma'].append(feature)
        elif any(keyword in feature for keyword in ['seasonal', 'trend', 'residual', 'factor']):
            feature_types['seasonal'].append(feature)
        else:
            feature_types['other'].append(feature)
    
    for ftype, features in feature_types.items():
        if features:
            print(f"  {ftype}: {len(features)}个特征")
    
    return available_features, filtered_features

# =============================================================================
# 特征选择函数 - 使用分位数回归森林结合其他算法
# =============================================================================
def select_features_with_qrf(X, y, product_name, output_dir):
    """
    使用分位数回归森林(QRF)结合其他算法进行特征选择
    返回最佳特征子集
    """
    print(f"\n开始为{product_name}执行特征选择...")
    
    # 创建特征选择输出目录
    feature_selection_dir = os.path.join(output_dir, "特征选择结果")
    os.makedirs(feature_selection_dir, exist_ok=True)
    print(f"特征选择结果将保存在: {feature_selection_dir}")
    
    # 1. 过滤可用特征
    available_features, filtered_features = filter_available_features(X.columns.tolist())
    
    if len(available_features) == 0:
        print(f"警告: {product_name}没有可用特征，无法进行特征选择")
        return []
    
    # 保存过滤结果
    filter_result_df = pd.DataFrame({
        'available_features': pd.Series(available_features),
        'filtered_features': pd.Series(filtered_features)
    })
    filter_result_path = os.path.join(feature_selection_dir, f'{product_name}_特征过滤结果.csv')
    filter_result_df.to_csv(filter_result_path, index=False)
    print(f"特征过滤结果已保存至: {filter_result_path}")
    
    # 2. 只使用可用特征
    X_available = X[available_features]
    
    # 3. 清理数据
    print("清理数据...")
    X_clean = clean_data(X_available.copy())
    y_clean = y.copy()
    
    # 确保索引一致
    y_clean = y_clean.loc[X_clean.index]

    # 4. 时间序列划分（使用清理后的数据）
    split_idx = int(len(X_clean) * 0.8)
    X_train = X_clean.iloc[:split_idx]
    X_test = X_clean.iloc[split_idx:]
    y_train = y_clean.iloc[:split_idx]
    y_test = y_clean.iloc[split_idx:]
    
    print(f"清理后训练集形状: {X_train.shape}, 测试集形状: {X_test.shape}")
    
    # 5. 使用分位数回归森林(QRF)进行特征重要性评估
    print("使用分位数回归森林(QRF)评估特征重要性...")
    qrf = RandomForestQuantileRegressor(
        n_estimators=300,
        min_samples_leaf=5,
        max_depth=10,
        n_jobs=-1,
        random_state=42
    )
    qrf.fit(X_train, y_train)
    
    # 获取特征重要性
    qrf_importances = pd.Series(
        qrf.feature_importances_, 
        index=X_clean.columns
    ).sort_values(ascending=False)
    
    # 可视化QRF特征重要性
    plt.figure(figsize=(16, 12))
    qrf_importances.head(30).plot(kind='barh')
    plt.title(f'{product_name} - QRF特征重要性 (Top 30, 仅可用特征)')
    plt.xlabel('重要性分数')
    plt.ylabel('特征')
    plt.tight_layout()
    plt.savefig(os.path.join(feature_selection_dir, f'{product_name}_QRF特征重要性.png'), dpi=300)
    plt.close()
    
    # 6. 使用递归特征消除(RFE)结合QRF
    print("使用递归特征消除(RFE)结合QRF选择特征...")
    n_features_to_select = min(20, len(available_features))  # 不超过可用特征数量
    rfe_qrf = RFE(
        estimator=clone(qrf),
        n_features_to_select=n_features_to_select,
        step=max(1, len(available_features) // 10),  # 动态调整步长
        verbose=1
    )
    rfe_qrf.fit(X_train, y_train)
    
    # 获取RFE选择的特征
    rfe_qrf_features = X_clean.columns[rfe_qrf.support_].tolist()
    print(f"RFE-QRF选择的特征数量: {len(rfe_qrf_features)}")
    
    # 7. 使用其他算法评估特征重要性
    algorithms = {
        "XGBoost": XGBRegressor(random_state=42, verbosity=0),
        "LightGBM": lgb.LGBMRegressor(random_state=42, verbosity=-1),
        "RandomForest": RandomForestRegressor(random_state=42),
        "GradientBoosting": GradientBoostingRegressor(random_state=42)
    }
    
    algorithm_importances = {}
    algorithm_features = {}
    
    for name, model in algorithms.items():
        print(f"使用{name}评估特征重要性...")
        model.fit(X_train, y_train)
        
        # 获取特征重要性
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
        elif hasattr(model, 'coef_'):
            importances = np.abs(model.coef_)
        else:
            importances = np.zeros(X_train.shape[1])
        
        # 存储重要性
        algorithm_importances[name] = pd.Series(
            importances, 
            index=X_clean.columns
        ).sort_values(ascending=False)
        
        # 可视化
        plt.figure(figsize=(16, 12))
        algorithm_importances[name].head(30).plot(kind='barh')
        plt.title(f'{product_name} - {name}特征重要性 (Top 30, 仅可用特征)')
        plt.xlabel('重要性分数')
        plt.ylabel('特征')
        plt.tight_layout()
        plt.savefig(os.path.join(feature_selection_dir, f'{product_name}_{name}特征重要性.png'), dpi=300)
        plt.close()
        
        # 使用SelectFromModel选择特征
        selector = SelectFromModel(
            model, 
            threshold="median",  # 选择重要性高于中位数的特征
            prefit=True
        )
        selected_features = X_clean.columns[selector.get_support()].tolist()
        algorithm_features[name] = selected_features
        print(f"{name}选择的特征数量: {len(selected_features)}")
    
    # 8. 综合所有方法选择最佳特征
    print("综合所有方法选择最佳特征...")
    
    # 创建特征得分表
    feature_scores = pd.DataFrame(index=X_clean.columns)
    
    # 添加QRF重要性得分
    feature_scores['QRF'] = qrf_importances
    
    # 添加其他算法的重要性得分
    for name in algorithms.keys():
        feature_scores[name] = algorithm_importances[name]
    
    # 标准化得分 (0-1范围)
    feature_scores = (feature_scores - feature_scores.min()) / (feature_scores.max() - feature_scores.min())
    
    # 计算平均得分
    feature_scores['Average'] = feature_scores.mean(axis=1)
    
    # 按平均得分排序
    feature_scores = feature_scores.sort_values('Average', ascending=False)
    
    # 保存特征得分
    feature_scores.to_csv(os.path.join(feature_selection_dir, f'{product_name}_特征得分.csv'))

    def evaluate_features_with_multiple_models(X_train_sel, X_test_sel, y_train, y_test):
        """
        使用多个模型评估特征集合的性能，返回平均RMSE
        """
        models = {
            'QRF': RandomForestQuantileRegressor(n_estimators=300,min_samples_leaf=5, max_depth=10,n_jobs=-1,random_state=42),
            'RandomForest': RandomForestRegressor(random_state=42),
            'XGBoost': XGBRegressor(random_state=42, verbosity=0),
            'LightGBM': lgb.LGBMRegressor(random_state=42, verbosity=-1),
            "GradientBoosting": GradientBoostingRegressor(random_state=42)
        }
        
        rmse_scores = []
        for name, model in models.items():
            try:
                model.fit(X_train_sel, y_train)
                y_pred = model.predict(X_test_sel)
                rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                rmse_scores.append(rmse)
            except Exception as e:
                print(f"    {name}模型评估失败: {str(e)}")
                continue
        
        # 返回平均RMSE
        return np.mean(rmse_scores) if rmse_scores else float('inf')

    # 9. 基于性能选择最佳特征数量
    print("基于模型性能选择最佳特征数量...")
    
    # 测试不同特征数量的性能
    max_features = min(50, len(feature_scores))
    results = []
    best_score = float('inf')
    best_features = []
    
    for n_features in range(5, max_features, 5):
        if n_features > len(feature_scores):
            break
            
        # 选择前n_features个特征
        selected_features = feature_scores.index[:n_features].tolist()
        X_train_sel = X_train[selected_features]
        X_test_sel = X_test[selected_features]
        
        avg_rmse = evaluate_features_with_multiple_models(
            X_train_sel, X_test_sel, y_train, y_test
        )
        
        # 存储结果
        results.append({
            'n_features': n_features,
            'avg_rmse': avg_rmse, 
            'features': selected_features
        })
        
        print(f"    {n_features}个特征的平均RMSE: {avg_rmse:.4f}")
        
        # 更新最佳特征
        if avg_rmse < best_score:
            best_score = avg_rmse
            best_features = selected_features
    
    # 将结果转换为DataFrame
    results_df = pd.DataFrame(results)
    
    # 可视化特征数量与性能的关系
    plt.figure(figsize=(12, 8))
    plt.plot(results_df['n_features'], results_df['avg_rmse'], 'o-')
    plt.title(f'{product_name} - 特征数量与模型性能 (仅可用特征)')
    plt.xlabel('特征数量')
    plt.ylabel('平均RMSE')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(feature_selection_dir, f'{product_name}_特征数量与性能.png'), dpi=300)
    plt.close()
    
    # 保存结果
    results_df.to_csv(os.path.join(feature_selection_dir, f'{product_name}_特征选择结果.csv'), index=False)
    
    print(f"最佳特征数量: {len(best_features)}, 平均RMSE: {best_score:.4f}")
    print(f"最佳特征列表:")
    for i, feature in enumerate(best_features, 1):
        print(f"  {i}. {feature}")
    
    return best_features

# =============================================================================
# 主程序
# =============================================================================

# 输入输出路径
output_dir = r"/Users/bytedance/Desktop/特征工程补充/结果"

# 处理三个品种
products = ['电解镍', '高碳铬铁', '铝锭']

for product in products:
    print(f"\n{'='*80}")
    print(f"开始处理 {product} 的特征选择")
    print(f"{'='*80}")
    
    # 创建产品输出目录
    product_output_dir = os.path.join(output_dir, f"{product}预测结果")
    os.makedirs(product_output_dir, exist_ok=True)
    print(f"输出目录: {product_output_dir}")
    
    # 1. 读取特征工程后的数据
    feature_path = os.path.join(product_output_dir, f"{product}_特征工程数据.xlsx")
    if not os.path.exists(feature_path):
        print(f"警告: 找不到 {product} 的特征工程数据文件: {feature_path}")
        continue
    
    feature_df = pd.read_excel(feature_path)
    
    # 检查数据是否为空
    if feature_df.empty:
        print(f"警告: {product} 的特征工程数据为空，跳过处理")
        continue
    
    # 2. 准备特征和目标变量
    # 排除非特征列（包括不能当天获得的特征）
    exclude_cols = [
        'date', 'demand', 'next_month_demand', 'holiday_name', 'sales',
        # 以下特征也应该排除，因为是当天无法获得的
        'avg_price', 'inquiry_count', 'our_quote_avg', 'inquiry_quantity', 
        'price_std', 'big_customer_inquiries', 'big_customer_quantity', 
        'big_customer_ratio', 'big_customer_quantity_ratio'
    ]
    
    feature_cols = [col for col in feature_df.columns if col not in exclude_cols]
    
    print(f"初始候选特征数量: {len(feature_cols)}")
    
    X = feature_df[feature_cols]
    y = feature_df['demand']
    
    # 3. 执行特征选择（包含特征过滤）
    best_features = select_features_with_qrf(X, y, product, product_output_dir)
    
    if not best_features:
        print(f"警告: {product}没有选出最佳特征，跳过后续处理")
        continue
    
    # 4. 保存最佳特征子集
    best_features_df = pd.DataFrame({'feature': best_features})
    best_features_path = os.path.join(product_output_dir, f"{product}_最佳特征.csv")
    best_features_df.to_csv(best_features_path, index=False)
    print(f"最佳特征已保存至: {best_features_path}")
    
    # 5. 使用最佳特征创建新数据集
    selected_features_df = feature_df[['date', 'demand'] + best_features]
    selected_features_path = os.path.join(product_output_dir, f"{product}_选定特征数据.xlsx")
    selected_features_df.to_excel(selected_features_path, index=False)
    print(f"选定特征数据集已保存至: {selected_features_path}")
    
    print(f"\n{product} 的特征选择完成!")

print("\n所有处理完成!")


开始处理 电解镍 的特征选择
输出目录: /Users/bytedance/Desktop/特征工程补充/结果/电解镍预测结果
初始候选特征数量: 235

开始为电解镍执行特征选择...
特征选择结果将保存在: /Users/bytedance/Desktop/特征工程补充/结果/电解镍预测结果/特征选择结果
开始过滤特征，排除实际应用中当天无法获得的特征...
原始特征数量: 235
可用特征数量: 235
被过滤特征数量: 0
保留的特征类型包括:
  time: 79个特征
  lag: 45个特征
  ma: 61个特征
  seasonal: 8个特征
  other: 42个特征
特征过滤结果已保存至: /Users/bytedance/Desktop/特征工程补充/结果/电解镍预测结果/特征选择结果/电解镍_特征过滤结果.csv
清理数据...
执行数据清理...
删除了 0 行缺失数据。
清理后训练集形状: (1216, 235), 测试集形状: (305, 235)
使用分位数回归森林(QRF)评估特征重要性...
使用递归特征消除(RFE)结合QRF选择特征...
Fitting estimator with 235 features.
Fitting estimator with 212 features.
Fitting estimator with 189 features.
Fitting estimator with 166 features.
Fitting estimator with 143 features.
Fitting estimator with 120 features.
Fitting estimator with 97 features.
Fitting estimator with 74 features.
Fitting estimator with 51 features.
Fitting estimator with 28 features.
RFE-QRF选择的特征数量: 20
使用XGBoost评估特征重要性...
XGBoost选择的特征数量: 118
使用LightGBM评估特征重要性...
LightGBM选择的特征数量: 126
使用RandomForest评估特征重要性...
Random

### 4. 预测

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import holidays
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, PowerTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor
import lightgbm as lgb
from quantile_forest import RandomForestQuantileRegressor
import warnings
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import uniform, randint
import time

# A/B 特征与测试期B类填充工具
from ab_feature_utils import BFeatureFillerDaily, separate_a_b_features

# 深度学习模型导入
try:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    import tensorflow as tf
    tf.get_logger().setLevel('ERROR')  # 减少TensorFlow日志
    KERAS_AVAILABLE = True
except ImportError:
    print("警告: TensorFlow/Keras未安装，LSTM、CNN模型将不可用")
    KERAS_AVAILABLE = False

# ARIMAX模型导入
try:
    from statsmodels.tsa.arima.model import ARIMA
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    STATSMODELS_AVAILABLE = True
except ImportError:
    print("警告: statsmodels未安装，ARIMAX模型将不可用")
    STATSMODELS_AVAILABLE = False


# =============================================================================
# 辅助函数
# =============================================================================
def calculate_streamlined_metrics(y_true, y_pred):
    """计算精简的评估指标"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # 基础指标
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    # 原始MAPE（处理除零问题）
    epsilon = 1e-10
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100
    
    # 加权MAPE（主要业务指标）
    weights = y_true / (y_true.sum() + epsilon)
    weighted_ape = np.abs((y_true - y_pred) / (y_true + epsilon)) * weights
    wmape = weighted_ape.sum() * 100
    
    return {
        'MAE': mae, 
        'RMSE': rmse, 
        'R²': r2, 
        'MAPE (%)': mape,  # 改为原始MAPE
        'WMAPE (%)': wmape
    }

def select_model_specific_features(features, X_train, y_train, model_type, model_name):
    """根据模型类型选择最适合的特征数量"""
    from sklearn.feature_selection import SelectKBest, f_regression, RFE
    from sklearn.feature_selection import mutual_info_regression
    from sklearn.linear_model import LinearRegression
    from sklearn.ensemble import RandomForestRegressor
    
    if model_type == 'linear':
        k = min(12, max(8, len(features) // 3))
        selector = SelectKBest(score_func=f_regression, k=k)
        X_selected = selector.fit_transform(X_train, y_train)
        selected_mask = selector.get_support()
        selected_features = [features[i] for i in range(len(features)) if selected_mask[i]]
        
        if len(selected_features) > 10:
            estimator = LinearRegression()
            rfe = RFE(estimator, n_features_to_select=10)
            rfe.fit(X_selected, y_train)
            final_mask = rfe.get_support()
            selected_features = [selected_features[i] for i in range(len(selected_features)) if final_mask[i]]
        
        return selected_features
    
    elif model_type == 'tree':
        k = min(len(features), max(20, int(len(features) * 0.8)))
        rf_temp = RandomForestRegressor(n_estimators=100, random_state=0)
        rf_temp.fit(X_train, y_train)
        
        importance_scores = rf_temp.feature_importances_
        feature_importance = list(zip(features, importance_scores))
        feature_importance.sort(key=lambda x: x[1], reverse=True)
        
        selected_features = [feat[0] for feat in feature_importance[:k]]
        return selected_features
    
    elif model_type in ['svm', 'neural']:
        k = min(15, max(10, len(features) // 2))
        selector = SelectKBest(score_func=mutual_info_regression, k=k)
        selector.fit(X_train, y_train)
        selected_mask = selector.get_support()
        selected_features = [features[i] for i in range(len(features)) if selected_mask[i]]
        return selected_features
    
    elif model_type == 'deep':
        k = min(10, max(6, len(features) // 4))
        selector = SelectKBest(score_func=f_regression, k=k)
        selector.fit(X_train, y_train)
        selected_mask = selector.get_support()
        selected_features = [features[i] for i in range(len(features)) if selected_mask[i]]
        return selected_features
    
    else:
        return features

def get_preprocessed_data(X_train, X_test, y_train, model_type='tree'):
    """根据模型类型选择合适的预处理方法"""
    
    if model_type == 'tree':
        scaler_X = RobustScaler()
        scaler_y = None
        
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)
        y_train_scaled = y_train.copy()
        
    elif model_type == 'neural':
        scaler_X = StandardScaler()
        try:
            scaler_y = PowerTransformer(method='yeo-johnson')
            y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
        except:
            scaler_y = StandardScaler()
            y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
        
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)
        
    elif model_type == 'svm':
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)
        y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    
    elif model_type == 'linear':
        scaler_X = StandardScaler()
        scaler_y = None
        
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)
        y_train_scaled = y_train.copy()
        
    elif model_type == 'deep':
        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()
        
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)
        y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    
    return X_train_scaled, X_test_scaled, y_train_scaled, scaler_X, scaler_y

def leak_free_fix_data_issues(train_df, test_df=None):
    """无数据泄漏的数据修复方法"""
    print("执行无数据泄漏的数据清理...")
    
    stats = {}
    train_cleaned = train_df.copy()
    
    # 1. 处理负值需求
    if 'demand' in train_cleaned.columns:
        min_demand = train_cleaned['demand'].min()
        if min_demand < 0:
            print(f"  修复负值需求: 最小值={min_demand}")
            train_cleaned['demand'] = train_cleaned['demand'].clip(lower=0)
    
    # 2. 处理无穷大值
    numeric_cols = train_cleaned.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if col == 'date':
            continue
            
        inf_count = np.isinf(train_cleaned[col]).sum()
        if inf_count > 0:
            finite_values = train_cleaned[col][np.isfinite(train_cleaned[col])]
            if len(finite_values) > 0:
                fill_value = finite_values.median()
            else:
                fill_value = 0
            stats[f'{col}_inf_fill'] = fill_value
            
            train_cleaned[col] = train_cleaned[col].replace([np.inf, -np.inf], fill_value)
    
    # 3. 处理极值
    for col in numeric_cols:
        if col in ['date', 'demand']:
            continue
            
        finite_values = train_cleaned[col][np.isfinite(train_cleaned[col])]
        if len(finite_values) > 10:
            Q1 = finite_values.quantile(0.25)
            Q3 = finite_values.quantile(0.75)
            IQR = Q3 - Q1
            
            if IQR > 0:
                lower_bound = Q1 - 3 * IQR
                upper_bound = Q3 + 3 * IQR
                stats[f'{col}_lower_bound'] = lower_bound
                stats[f'{col}_upper_bound'] = upper_bound
                
                extreme_mask = (train_cleaned[col] < lower_bound) | (train_cleaned[col] > upper_bound)
                extreme_count = extreme_mask.sum()
                if extreme_count > 0:
                    train_cleaned.loc[extreme_mask, col] = np.clip(train_cleaned[col], lower_bound, upper_bound)
        else:
            stats[f'{col}_lower_bound'] = -1e10
            stats[f'{col}_upper_bound'] = 1e10
    
    # 4. 处理NaN值
    for col in train_cleaned.columns:
        if train_cleaned[col].isna().any():
            if col == 'date':
                train_cleaned = train_cleaned.dropna(subset=['date'])
            elif col == 'demand':
                train_cleaned[col] = train_cleaned[col].fillna(method='ffill').fillna(0)
            elif train_cleaned[col].dtype == 'object':
                train_cleaned[col] = train_cleaned[col].fillna('unknown')
            else:
                fill_value = train_cleaned[col].median() if train_cleaned[col].notna().sum() > 0 else 0
                stats[f'{col}_nan_fill'] = fill_value
                train_cleaned[col] = train_cleaned[col].fillna(method='ffill').fillna(fill_value)
    
    # 5. 确保日期列有效
    if 'date' in train_cleaned.columns:
        train_cleaned['date'] = pd.to_datetime(train_cleaned['date'], errors='coerce')
        train_cleaned = train_cleaned.dropna(subset=['date'])
    
    # 如果提供了测试集，使用训练集统计量处理测试集
    if test_df is not None:
        test_cleaned = test_df.copy()
        
        # 使用相同的处理逻辑，但用训练集的统计量
        if 'demand' in test_cleaned.columns:
            test_cleaned['demand'] = test_cleaned['demand'].clip(lower=0)
        
        for col in numeric_cols:
            if col == 'date':
                continue
                
            inf_count = np.isinf(test_cleaned[col]).sum()
            if inf_count > 0:
                fill_value = stats.get(f'{col}_inf_fill', 0)
                test_cleaned[col] = test_cleaned[col].replace([np.inf, -np.inf], fill_value)
        
        for col in numeric_cols:
            if col in ['date', 'demand']:
                continue
                
            lower_bound = stats.get(f'{col}_lower_bound', -1e10)
            upper_bound = stats.get(f'{col}_upper_bound', 1e10)
            
            extreme_mask = (test_cleaned[col] < lower_bound) | (test_cleaned[col] > upper_bound)
            extreme_count = extreme_mask.sum()
            if extreme_count > 0:
                test_cleaned.loc[extreme_mask, col] = np.clip(test_cleaned[col], lower_bound, upper_bound)
        
        for col in test_cleaned.columns:
            if test_cleaned[col].isna().any():
                if col == 'date':
                    test_cleaned = test_cleaned.dropna(subset=['date'])
                elif col == 'demand':
                    test_cleaned[col] = test_cleaned[col].fillna(method='ffill').fillna(0)
                elif test_cleaned[col].dtype == 'object':
                    test_cleaned[col] = test_cleaned[col].fillna('unknown')
                else:
                    fill_value = stats.get(f'{col}_nan_fill', 0)
                    test_cleaned[col] = test_cleaned[col].fillna(method='ffill').fillna(fill_value)
        
        if 'date' in test_cleaned.columns:
            test_cleaned['date'] = pd.to_datetime(test_cleaned['date'], errors='coerce')
            test_cleaned = test_cleaned.dropna(subset=['date'])
        
        return train_cleaned, test_cleaned
    
    else:
        return train_cleaned, stats

def load_selected_features(product, output_dir):
    """加载已选择的最佳特征"""
    product_output_dir = os.path.join(output_dir, f"{product}预测结果")
    best_features_path = os.path.join(product_output_dir, f"{product}_最佳特征.csv")
    
    if os.path.exists(best_features_path):
        best_features_df = pd.read_csv(best_features_path)
        return best_features_df['feature'].tolist()
    else:
        print(f"警告: 找不到{product}的最佳特征文件，使用默认特征")
        return [
            'is_holiday', 'is_Monday', 'is_Tuesday', 'is_Wednesday', 'is_Thursday', 
            'is_Friday', 'is_Saturday', 'is_Sunday', 'is_weekend', 'demand_lag14', 
            'demand_ma3', 'demand_ma7', 'seasonal_factor', 'weekday_factor', 
            'is_future_volatile', 'trend', 'residual', 'seasonal'
        ]

# =============================================================================
# 核心类：逐日预测器
# =============================================================================
class DailyPredictor:
    """逐日预测器类 - 模拟实际应用场景"""
    
    def __init__(self, model, scaler_X, scaler_y, features, model_type, model_name):
        self.model = model
        self.scaler_X = scaler_X
        self.scaler_y = scaler_y
        self.features = features
        self.model_type = model_type
        self.model_name = model_name
        self.prediction_history = []
    
    def predict_single_day(self, daily_features_dict):
        """预测单天需求"""
        try:
            # 构造特征向量
            feature_vector = []
            for feature in self.features:
                value = daily_features_dict.get(feature, 0)
                if pd.isna(value) or np.isinf(value):
                    value = 0
                feature_vector.append(value)
            
            feature_vector = np.array(feature_vector).reshape(1, -1)
            
            # 预处理
            feature_scaled = self.scaler_X.transform(feature_vector)
            
            # 预测
            if self.model_type == 'deep':
                pred_scaled = self.model.predict(feature_scaled, verbose=0)[0]
            else:
                pred_scaled = self.model.predict(feature_scaled)[0]
            
            # 逆变换
            if self.scaler_y is not None:
                pred = self.scaler_y.inverse_transform([[pred_scaled]])[0, 0]
            else:
                pred = pred_scaled
            
            # 确保非负
            pred = max(0, pred)
            
            # 记录预测历史
            self.prediction_history.append(pred)
            
            return pred
            
        except Exception as e:
            print(f"  {self.model_name} 单日预测失败: {str(e)}")
            return 0
    
    def predict_test_period(self, test_data, train_data=None):
        """逐日预测整个测试期间（测试期仅使用A类特征+预测填充的B类特征）。"""
        # 兼容：未提供训练数据时，退回原逻辑
        if train_data is None:
            predictions = []
            for i in range(len(test_data)):
                daily_features = test_data.iloc[i][self.features].to_dict()
                pred = self.predict_single_day(daily_features)
                predictions.append(pred)
            return np.array(predictions)

        try:
            test_sorted = test_data.copy()
            if 'date' in test_sorted.columns:
                test_sorted = test_sorted.sort_values('date').reset_index(drop=True)

            a_feats, b_feats = separate_a_b_features(self.features)

            filler = BFeatureFillerDaily(
                train_df=train_data[['date', 'demand'] + [c for c in self.features if c in train_data.columns]].copy(),
                features=self.features,
                a_features=a_feats,
                b_features=b_feats,
                date_col='date',
            )

            predictions = []
            for i in range(len(test_sorted)):
                row = test_sorted.iloc[i]
                feat_dict = filler.build_features_for_row(row)
                pred = self.predict_single_day(feat_dict)
                predictions.append(pred)
                filler.update_with_prediction(pred)

            return np.array(predictions)
        except Exception as e:
            print(f"  {self.model_name} 使用A/B填充预测失败，退回原逻辑: {str(e)}")
            predictions = []
            for i in range(len(test_data)):
                daily_features = test_data.iloc[i][self.features].to_dict()
                pred = self.predict_single_day(daily_features)
                predictions.append(pred)
            return np.array(predictions)

class SequencePredictor:
    """序列预测器类 - 用于LSTM/GRU等需要序列数据的模型"""
    
    def __init__(self, model, scaler_X, scaler_y, features, sequence_length, model_name):
        self.model = model
        self.scaler_X = scaler_X
        self.scaler_y = scaler_y
        self.features = features
        self.sequence_length = sequence_length
        self.model_name = model_name
        self.feature_history = []  # 存储特征历史
    
    def add_daily_features(self, daily_features_dict):
        """添加当天特征到历史记录"""
        feature_vector = []
        for feature in self.features:
            value = daily_features_dict.get(feature, 0)
            if pd.isna(value) or np.isinf(value):
                value = 0
            feature_vector.append(value)
        
        # 预处理特征
        feature_scaled = self.scaler_X.transform([feature_vector])[0]
        
        # 添加到历史
        self.feature_history.append(feature_scaled)
        
        # 保持序列长度
        if len(self.feature_history) > self.sequence_length:
            self.feature_history.pop(0)
    
    def predict_single_day(self):
        """基于历史序列预测当天需求"""
        try:
            if len(self.feature_history) < self.sequence_length:
                return 0  # 历史数据不足
            
            # 构造序列
            sequence = np.array(self.feature_history[-self.sequence_length:]).reshape(1, self.sequence_length, -1)
            
            # 预测
            pred_scaled = self.model.predict(sequence, verbose=0)[0, 0]
            
            # 逆变换
            if self.scaler_y is not None:
                pred = self.scaler_y.inverse_transform([[pred_scaled]])[0, 0]
            else:
                pred = pred_scaled
            
            return max(0, pred)
            
        except Exception as e:
            print(f"  {self.model_name} 序列预测失败: {str(e)}")
            return 0
    
    def predict_test_period(self, test_data, train_data):
        """逐日预测整个测试期间"""
        predictions = []
        
        # 用训练数据的最后几天初始化历史
        if len(train_data) >= self.sequence_length:
            for i in range(-self.sequence_length, 0):
                daily_features = train_data.iloc[i][self.features].to_dict()
                self.add_daily_features(daily_features)
        
        # 逐日预测测试数据
        for i in range(len(test_data)):
            # 预测当天
            pred = self.predict_single_day()
            predictions.append(pred)
            
            # 添加当天特征到历史（用于下一天预测）
            daily_features = test_data.iloc[i][self.features].to_dict()
            self.add_daily_features(daily_features)
        
        return np.array(predictions)

# =============================================================================
# 改进的集成和评估函数
# =============================================================================
def create_performance_based_ensemble(model_predictions, model_performance, exclude_baselines=True):
    """基于性能创建集成预测，排除基线模型"""
    
    # 排除基线模型
    baseline_models = ['LinearRegression', '13周移动平均'] if exclude_baselines else []
    
    # 过滤模型：排除基线模型和表现极差的模型
    filtered_predictions = {}
    valid_performances = {}
    
    for name, pred in model_predictions.items():
        if name not in baseline_models and name in model_performance:
            # 排除WMAPE > 80%的模型（表现极差）
            wmape = model_performance[name].get('WMAPE (%)', float('inf'))
            if wmape < 80:
                filtered_predictions[name] = pred
                valid_performances[name] = model_performance[name]
    
    if len(filtered_predictions) < 2:
        print("可用于集成的模型数量不足，使用所有非基线模型")
        # 如果过滤后模型太少，放宽条件
        for name, pred in model_predictions.items():
            if name not in baseline_models and name in model_performance:
                filtered_predictions[name] = pred
                valid_performances[name] = model_performance[name]
    
    if len(filtered_predictions) < 2:
        return None, None
    
    # 基于多指标计算权重
    def calculate_composite_score(metrics):
        """计算综合得分"""
        wmape = metrics.get('WMAPE (%)', 100)
        mae = metrics.get('MAE', float('inf'))
        r2 = max(0, metrics.get('R²', 0))  # R²可能为负，设下限为0
        
        # 归一化分数（越小越好的指标转换为越大越好）
        wmape_score = 1.0 / (1.0 + wmape / 50)  # 50%为基准
        mae_score = 1.0 / (1.0 + mae / 50000)   # 根据实际MAE调整基准
        r2_score = r2
        
        # 加权组合：WMAPE(40%) + MAE(30%) + R²(30%)
        composite = 0.4 * wmape_score + 0.3 * mae_score + 0.3 * r2_score
        return composite
    
    # 计算权重
    scores = {name: calculate_composite_score(metrics) 
              for name, metrics in valid_performances.items()}
    
    total_score = sum(scores.values())
    if total_score == 0:
        # 如果所有得分都是0，使用等权重
        weights = {name: 1.0/len(scores) for name in scores.keys()}
    else:
        weights = {name: score / total_score for name, score in scores.items()}
    
    # 计算集成预测
    pred_lengths = [len(pred) for pred in filtered_predictions.values()]
    base_length = min(pred_lengths)
    
    ensemble_pred = np.zeros(base_length)
    
    print("基于性能的集成权重:")
    for model_name, pred in filtered_predictions.items():
        weight = weights[model_name]
        wmape = valid_performances[model_name]['WMAPE (%)']
        print(f"  {model_name}: {weight:.3f} (WMAPE: {wmape:.2f}%)")
        
        pred_adj = pred[:base_length]
        ensemble_pred += weight * pred_adj
    
    return np.maximum(ensemble_pred, 0), weights

def calculate_model_ranking(model_performance, exclude_baselines=True):
    """计算模型综合排名"""
    
    # 排除基线模型
    baseline_models = ['LinearRegression', '13周移动平均'] if exclude_baselines else []
    filtered_performance = {k: v for k, v in model_performance.items() if k not in baseline_models}
    
    if not filtered_performance:
        return []
    
    # 主要指标权重
    weights = {
        'WMAPE (%)': 0.4,    # 业务最关心的指标
        'MAE': 0.3,          # 直观的误差指标
        'R²': 0.3            # 拟合度指标
    }
    
    # 计算综合排名
    models = list(filtered_performance.keys())
    rankings = {}
    
    for metric, weight in weights.items():
        if metric == 'R²':
            # R²越大越好
            sorted_models = sorted(models, 
                                 key=lambda x: filtered_performance[x].get(metric, -1), 
                                 reverse=True)
        else:
            # 其他指标越小越好
            sorted_models = sorted(models, 
                                 key=lambda x: filtered_performance[x].get(metric, float('inf')))
        
        # 分配排名分数（第一名得分最高）
        for i, model in enumerate(sorted_models):
            if model not in rankings:
                rankings[model] = 0
            rankings[model] += weight * (len(models) - i)
    
    # 按综合得分排序
    final_ranking = sorted(rankings.items(), key=lambda x: x[1], reverse=True)
    
    print("\n基于WMAPE(40%)+MAE(30%)+R²(30%)的综合排名:")
    for i, (model, score) in enumerate(final_ranking, 1):
        wmape = filtered_performance[model].get('WMAPE (%)', 'N/A')
        mae = filtered_performance[model].get('MAE', 'N/A')
        r2 = filtered_performance[model].get('R²', 'N/A')
        if isinstance(wmape, (int, float)):
            wmape = f"{wmape:.2f}%"
        if isinstance(mae, (int, float)):
            mae = f"{mae:.0f}"
        if isinstance(r2, (int, float)):
            r2 = f"{r2:.4f}"
        print(f"  {i}. {model}: {score:.2f}分 (WMAPE: {wmape}, MAE: {mae}, R²: {r2})")
    
    return final_ranking


# 忽略警告
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS']

def create_improved_models():
    """创建改进的默认模型配置（用于非搜索模式）"""
    
    models_config = {
        "RandomForest": {
            'model': RandomForestRegressor(
                n_estimators=300,
                max_depth=12,
                min_samples_split=5,
                min_samples_leaf=2,
                max_features='sqrt',
                bootstrap=True,
                n_jobs=-1,
                random_state=0
            ),
            'type': 'tree'
        },
        "XGBoost": {
            'model': XGBRegressor(
                objective='reg:squarederror',
                n_estimators=500,
                learning_rate=0.02,
                max_depth=6,
                min_child_weight=1,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.1,
                reg_lambda=1.0,
                random_state=0,
                n_jobs=-1,
                verbosity=0
            ),
            'type': 'tree'
        },
        "LightGBM": {
            'model': lgb.LGBMRegressor(
                objective='regression',
                n_estimators=500,
                learning_rate=0.02,
                max_depth=6,
                num_leaves=31,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.1,
                reg_lambda=1.0,
                random_state=42,
                n_jobs=-1,
                verbose=-1
            ),
            'type': 'tree'
        },
        "QRF": {
            'model': RandomForestQuantileRegressor(
                n_estimators=300,
                max_depth=12,
                min_samples_leaf=2,
                max_features='sqrt',
                n_jobs=-1,
                random_state=42
            ),
            'type': 'tree'
        },
        "SVR": {
            'model': SVR(kernel='rbf', C=1000, gamma='auto', epsilon=0.01),
            'type': 'svm'
        }
    }
    
    return models_config

def create_unified_deep_models(input_shape):
    """创建统一的深度学习模型（仅LSTM）"""
    
    def create_lstm_model(input_shape):
        model = Sequential([
            LSTM(32, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(16, return_sequences=False),
            Dropout(0.2),
            Dense(8, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mae'])
        return model
    
    deep_models = {
        "LSTM": create_lstm_model(input_shape)
    }
    
    return deep_models

def get_hyperparameter_search_config():
    """获取超参数搜索配置"""
    
    models_config = {
        "RandomForest": {
            'base_model': RandomForestRegressor(n_jobs=-1, random_state=0),
            'param_grid': {
                'n_estimators': [200, 300, 500],
                'max_depth': [10, 12, 15],
                'min_samples_split': [3, 5, 8],
                'min_samples_leaf': [2, 4]
            },
            'param_distributions': {
                'n_estimators': randint(200, 600),
                'max_depth': randint(8, 16),
                'min_samples_split': randint(2, 10),
                'min_samples_leaf': randint(1, 5)
            },
            'type': 'tree',
            'n_iter': 20
        },
        
        "XGBoost": {
            'base_model': XGBRegressor(
                objective='reg:squarederror',
                random_state=0,
                n_jobs=-1,
                verbosity=0
            ),
            'param_grid': {
                'n_estimators': [400, 600, 800],
                'learning_rate': [0.01, 0.02, 0.05, 0.08],
                'max_depth': [5, 6, 7, 8],
                'min_child_weight': [1, 3, 5],
                'subsample': [0.7, 0.8, 0.9],
                'colsample_bytree': [0.7, 0.8, 0.9]
            },
            'param_distributions': {
                'n_estimators': randint(300, 1000),
                'learning_rate': uniform(0.005, 0.095),
                'max_depth': randint(4, 12),
                'min_child_weight': randint(1, 8),
                'subsample': uniform(0.65, 0.3),
                'colsample_bytree': uniform(0.65, 0.3),
                'gamma': uniform(0, 0.5),
                'reg_alpha': uniform(0, 0.2),
                'reg_lambda': uniform(0.5, 2.5)
            },
            'type': 'tree',
            'n_iter': 30
        },
        
        "LightGBM": {
            'base_model': lgb.LGBMRegressor(
                objective='regression',
                random_state=42,
                n_jobs=-1,
                verbose=-1
            ),
            'param_grid': {
                'n_estimators': [400, 600, 800],
                'learning_rate': [0.01, 0.02, 0.05],
                'max_depth': [5, 6, 8],
                'num_leaves': [31, 50, 70],
                'subsample': [0.7, 0.8, 0.9],
                'colsample_bytree': [0.7, 0.8, 0.9]
            },
            'param_distributions': {
                'n_estimators': randint(300, 1000),
                'learning_rate': uniform(0.005, 0.095),
                'max_depth': randint(4, 12),
                'num_leaves': randint(20, 80),
                'subsample': uniform(0.65, 0.3),
                'colsample_bytree': uniform(0.65, 0.3),
                'reg_alpha': uniform(0, 0.2),
                'reg_lambda': uniform(0.5, 2.5)
            },
            'type': 'tree',
            'n_iter': 30
        },
        
        "QRF": {
            'base_model': RandomForestQuantileRegressor(n_jobs=-1, random_state=42),
            'param_grid': {
                'n_estimators': [200, 300],
                'max_depth': [10, 12],
                'min_samples_leaf': [2, 4]
            },
            'param_distributions': {
                'n_estimators': randint(200, 500),
                'max_depth': randint(8, 15),
                'min_samples_leaf': randint(1, 5)
            },
            'type': 'tree',
            'n_iter': 15
        },
        
        "SVR": {
            'base_model': SVR(),
            'param_grid': {
                'kernel': ['rbf'],
                'C': [100, 2000],  
                'gamma': ['scale', 0.04], 
                'epsilon': [0.05, 0.1]  
            },
            'type': 'svm',
            'use_grid_only': True
        }
    }
    
    return models_config

def unified_deep_learning_training_with_search(train_cleaned, test_cleaned, working_features, y_train, y_test, 
                                  model_performance, daily_predictors, sequence_predictors, product_name,
                                  enable_search=True):
    """统一的深度学习模型训练（仅LSTM，支持超参数搜索）"""
    
    if not KERAS_AVAILABLE:
        return
        
    print(f"\n训练深度学习模型{'（启用搜索）' if enable_search else ''}...")
    
    # 为深度学习模型选择特征
    deep_features = select_model_specific_features(
        working_features,
        train_cleaned[working_features],
        y_train,
        'deep',
        'DeepLearning'
    )
    
    print(f"深度学习模型使用 {len(deep_features)} 个特征")
    
    X_train_deep_raw = train_cleaned[deep_features]
    X_test_deep_raw = test_cleaned[deep_features]
    
    # 使用统一的预处理
    X_train_deep, X_test_deep, y_train_deep, scaler_X_deep, scaler_y_deep = get_preprocessed_data(
        X_train_deep_raw, X_test_deep_raw, y_train, 'deep'
    )
    
    # 使用统一的序列长度
    sequence_length = 7
    max_possible_length = len(X_train_deep) // 8
    sequence_length = min(sequence_length, max(3, max_possible_length))
    
    print(f"使用序列长度: {sequence_length}")
    
    if len(X_train_deep) <= sequence_length * 3:
        print("训练数据不足，跳过深度学习模型")
        return
    
    # 准备序列训练数据
    def prepare_sequence_data(X, y, seq_len):
        X_seq, y_seq = [], []
        for i in range(seq_len, len(X)):
            X_seq.append(X[i-seq_len:i])
            y_seq.append(y[i])
        return np.array(X_seq), np.array(y_seq)
    
    X_train_seq, y_train_seq = prepare_sequence_data(X_train_deep, y_train_deep, sequence_length)
    
    if len(X_train_seq) <= 15:
        print("序列数据不足，跳过深度学习模型")
        return
    
    # 训练验证集分割
    val_size = max(5, len(X_train_seq) // 5)
    X_train_model = X_train_seq[:-val_size]
    y_train_model = y_train_seq[:-val_size]
    X_val_model = X_train_seq[-val_size:]
    y_val_model = y_train_seq[-val_size:]
    
    # 定义超参数搜索空间
    if enable_search:
        lstm_configs = [
            {'units1': 32, 'units2': 16, 'dropout': 0.2, 'lr': 0.01, 'dense': 8},
            {'units1': 24, 'units2': 12, 'dropout': 0.25, 'lr': 0.005, 'dense': 6},
            {'units1': 48, 'units2': 24, 'dropout': 0.2, 'lr': 0.01, 'dense': 12},
            {'units1': 32, 'units2': 16, 'dropout': 0.3, 'lr': 0.005, 'dense': 8},
            {'units1': 40, 'units2': 20, 'dropout': 0.2, 'lr': 0.008, 'dense': 10},
        ]
    else:
        # 默认配置
        lstm_configs = [
            {'units1': 32, 'units2': 16, 'dropout': 0.2, 'lr': 0.01, 'dense': 8}
        ]
    
    input_shape = (sequence_length, X_train_deep.shape[1])
    
    # 训练LSTM
    print(f"\n训练 LSTM 模型{'（搜索中）' if enable_search else ''}...")
    best_lstm_model = None
    best_lstm_score = float('inf')
    best_lstm_config = None
    
    for i, config in enumerate(lstm_configs):
        if enable_search:
            print(f"  尝试配置 {i+1}/{len(lstm_configs)}: {config}")
        
        try:
            model = Sequential([
                LSTM(config['units1'], return_sequences=True, input_shape=input_shape),
                Dropout(config['dropout']),
                LSTM(config['units2'], return_sequences=False),
                Dropout(config['dropout']),
                Dense(config['dense'], activation='relu'),
                Dense(1)
            ])
            model.compile(optimizer=Adam(learning_rate=config['lr']), loss='mse', metrics=['mae'])
            
            callbacks = [
                EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
            ]
            
            history = model.fit(
                X_train_model, y_train_model,
                validation_data=(X_val_model, y_val_model),
                epochs=50,
                batch_size=min(16, max(4, len(X_train_seq) // 8)),
                callbacks=callbacks,
                verbose=0
            )
            
            val_loss = min(history.history['val_loss'])
            
            if enable_search:
                print(f"    验证损失: {val_loss:.4f}")
            
            if val_loss < best_lstm_score:
                best_lstm_score = val_loss
                best_lstm_model = model
                best_lstm_config = config
                
        except Exception as e:
            print(f"  配置 {i+1} 训练失败: {str(e)}")
            continue
    
    if best_lstm_model is not None:
        if enable_search:
            print(f"  LSTM最佳配置: {best_lstm_config}, 验证损失: {best_lstm_score:.4f}")
        
        predictor = SequencePredictor(
            best_lstm_model, scaler_X_deep, scaler_y_deep, 
            deep_features, sequence_length, 'LSTM'
        )
        sequence_predictors['LSTM'] = predictor
        
        print(f"  执行 LSTM 逐日序列预测...")
        y_pred = predictor.predict_test_period(test_cleaned, train_cleaned)
        
        if len(y_pred) > 0:
            min_len = min(len(y_test), len(y_pred))
            y_test_adj = y_test.iloc[:min_len].values
            y_pred_adj = y_pred[:min_len]
            
            if min_len > 0:
                performance = calculate_streamlined_metrics(y_test_adj, y_pred_adj)
                model_performance['LSTM'] = performance
                print(f"  LSTM 逐日预测性能: MAE={performance['MAE']:.2f}, WMAPE={performance['WMAPE (%)']:.2f}%, R²={performance['R²']:.4f}")
                daily_predictors['LSTM'] = predictor

def train_models_with_hyperparameter_search(train_cleaned, working_features, y_train, product_name,
                                           max_time_per_model=180, search_strategy="smart"):
    """使用超参数搜索训练模型"""
    
    print(f"开始超参数搜索训练（策略: {search_strategy}）...")
    
    # 获取搜索配置
    models_config = get_hyperparameter_search_config()
    
    trained_models = {}
    best_params_dict = {}
    
    for name, config in models_config.items():
        print(f"\n训练 {name} 模型...")
        
        try:
            model_type = config['type']
            
            # 特征选择
            model_features = select_model_specific_features(
                working_features, 
                train_cleaned[working_features], 
                y_train, 
                model_type, 
                name
            )
            
            print(f"  {name} 使用 {len(model_features)} 个特征")
            
            X_train = train_cleaned[model_features]
            X_train_processed, _, y_train_processed, scaler_X, scaler_y = get_preprocessed_data(
                X_train, X_train, y_train, model_type
            )
            
            # 执行搜索
            best_model, best_params, best_score, search_time = smart_hyperparameter_search(
                config, X_train_processed, y_train_processed, name
            )
            
            # 存储结果
            trained_models[name] = {
                'model': best_model,
                'features': model_features,
                'scaler_X': scaler_X,
                'scaler_y': scaler_y,
                'type': model_type,
                'best_score': best_score,
                'search_time': search_time
            }
            
            best_params_dict[name] = best_params
            
            print(f"  {name} 搜索完成，用时: {search_time:.1f}秒")
            
        except Exception as e:
            print(f"  训练 {name} 模型时出错: {str(e)}")
            continue
    
    return trained_models, best_params_dict

def train_models_with_improved_defaults(train_cleaned, working_features, y_train, product_name):
    """使用改进默认参数的快速训练"""
    
    # 使用原来优化过的create_improved_models函数
    models_config = create_improved_models()
    
    trained_models = {}
    best_params_dict = {}
    
    for name, config in models_config.items():
        print(f"\n训练 {name} 模型（改进默认参数）...")
        
        try:
            model = config['model']
            model_type = config['type']
            
            model_features = select_model_specific_features(
                working_features, 
                train_cleaned[working_features], 
                y_train, 
                model_type, 
                name
            )
            
            X_train = train_cleaned[model_features]
            X_train_processed, _, y_train_processed, scaler_X, scaler_y = get_preprocessed_data(
                X_train, X_train, y_train, model_type
            )
            
            # 对SVR做网格搜索
            if name == 'SVR':
                print(f"  执行 {name} 网格搜索...")
                
                from sklearn.model_selection import GridSearchCV
                
                param_grid = {
                    'C': [100, 1000, 5000],
                    'gamma': ['scale', 'auto', 0.01],
                    'epsilon': [0.01, 0.05, 0.1]
                }
                
                tscv = TimeSeriesSplit(n_splits=3)
                grid_search = GridSearchCV(
                    model, param_grid, 
                    scoring='neg_mean_absolute_error',
                    cv=tscv, n_jobs=1, verbose=0
                )
                grid_search.fit(X_train_processed, y_train_processed)
                model = grid_search.best_estimator_
                
                print(f"  最佳参数: {grid_search.best_params_}")
                print(f"  最佳得分: {grid_search.best_score_:.2f}")
            else:
                # 其他模型正常训练
                if name in ['XGBoost', 'LightGBM']:
                    eval_set = [(X_train_processed, y_train_processed)]
                    if name == 'XGBoost':
                        model.fit(X_train_processed, y_train_processed, 
                                eval_set=eval_set, verbose=False)
                    else:  # LightGBM
                        model.fit(X_train_processed, y_train_processed, 
                                eval_set=eval_set, 
                                callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)])
                else:
                    model.fit(X_train_processed, y_train_processed)
            
            trained_models[name] = {
                'model': model,
                'features': model_features,
                'scaler_X': scaler_X,
                'scaler_y': scaler_y,
                'type': model_type,
                'best_score': None,
                'search_time': 0
            }
            
            # 获取当前参数作为"最佳参数"
            best_params_dict[name] = model.get_params()
            
            print(f"  {name} 训练完成")
            
        except Exception as e:
            print(f"  训练 {name} 模型时出错: {str(e)}")
            continue
    
    return trained_models, best_params_dict

def smart_hyperparameter_search(config, X_train, y_train, model_name, data_threshold=200):
    """智能超参数搜索：根据数据量和模型类型选择搜索策略"""
    
    data_size = len(X_train)
    print(f"    数据量: {data_size}")
    
    start_time = time.time()
    
    try:
        # 动态调整CV折数
        cv_folds = min(3, max(2, data_size // 20))
        tscv = TimeSeriesSplit(n_splits=cv_folds)
        
        # SVR强制使用网格搜索
        if config.get('use_grid_only', False):
            print(f"    使用网格搜索（强制模式）")
            
            search = GridSearchCV(
                estimator=config['base_model'],
                param_grid=config['param_grid'],
                scoring='neg_mean_absolute_error',
                cv=tscv,
                n_jobs=1,
                verbose=0
            )
        elif data_size < data_threshold:
            # 小数据集：使用网格搜索
            print(f"    使用网格搜索（数据量 < {data_threshold}）")
            
            search = GridSearchCV(
                estimator=config['base_model'],
                param_grid=config['param_grid'],
                scoring='neg_mean_absolute_error',
                cv=tscv,
                n_jobs=1,
                verbose=0,
                random_state=42
            )
        else:
            # 大数据集：使用随机搜索
            print(f"    使用随机搜索（数据量 >= {data_threshold}）")
            
            search = RandomizedSearchCV(
                estimator=config['base_model'],
                param_distributions=config['param_distributions'],
                n_iter=config['n_iter'],
                scoring='neg_mean_absolute_error',
                cv=tscv,
                n_jobs=1,
                verbose=0,
                random_state=42
            )
        
        search.fit(X_train, y_train)
        
        search_time = time.time() - start_time
        
        print(f"    搜索完成，用时: {search_time:.1f}秒")
        print(f"    最佳参数: {search.best_params_}")
        print(f"    最佳得分: {search.best_score_:.4f}")
        
        return search.best_estimator_, search.best_params_, search.best_score_, search_time
        
    except Exception as e:
        print(f"    智能搜索失败: {str(e)}")
        return fallback_to_defaults(config, X_train, y_train, model_name)

def grid_search_with_timeout(config, X_train, y_train, model_name, max_time=180):
    """带超时的网格搜索"""
    
    print(f"    执行网格搜索（最大时间: {max_time}秒）")
    
    start_time = time.time()
    
    try:
        tscv = TimeSeriesSplit(n_splits=3)
        
        search = GridSearchCV(
            estimator=config['base_model'],
            param_grid=config['param_grid'],
            scoring='neg_mean_absolute_error',
            cv=tscv,
            n_jobs=1,
            verbose=0
        )
        
        search.fit(X_train, y_train)
        
        search_time = time.time() - start_time
        
        return search.best_estimator_, search.best_params_, search.best_score_, search_time
        
    except Exception as e:
        print(f"    网格搜索失败: {str(e)}")
        return fallback_to_defaults(config, X_train, y_train, model_name)

def random_search_with_timeout(config, X_train, y_train, model_name, max_time=180):
    """带超时的随机搜索"""
    
    print(f"    执行随机搜索（最大时间: {max_time}秒）")
    
    start_time = time.time()
    
    try:
        tscv = TimeSeriesSplit(n_splits=3)
        
        search = RandomizedSearchCV(
            estimator=config['base_model'],
            param_distributions=config['param_distributions'],
            n_iter=config['n_iter'],
            scoring='neg_mean_absolute_error',
            cv=tscv,
            n_jobs=1,
            verbose=0,
            random_state=42
        )
        
        search.fit(X_train, y_train)
        
        search_time = time.time() - start_time
        
        return search.best_estimator_, search.best_params_, search.best_score_, search_time
        
    except Exception as e:
        print(f"    随机搜索失败: {str(e)}")
        return fallback_to_defaults(config, X_train, y_train, model_name)

def fallback_to_defaults(config, X_train, y_train, model_name):
    """fallback到默认参数"""
    
    print(f"    使用默认参数...")
    
    start_time = time.time()
    
    model = config['base_model']
    
    # 使用改进的默认参数
    if model_name == 'SVR':
        model.set_params(kernel='rbf', C=1000, gamma='auto', epsilon=0.01)
    elif model_name == 'XGBoost':
        model.set_params(n_estimators=500, learning_rate=0.02, max_depth=6)
    elif model_name == 'ANN':
        model.set_params(hidden_layer_sizes=(128, 64, 32), alpha=0.001, learning_rate_init=0.001)
    
    model.fit(X_train, y_train)
    
    search_time = time.time() - start_time
    
    return model, model.get_params(), None, search_time

def save_best_parameters(best_params_dict, product_name, output_dir):
    """保存最佳参数到文件"""
    
    if not best_params_dict:
        return
    
    import pandas as pd
    
    params_data = []
    for model_name, params in best_params_dict.items():
        for param_name, param_value in params.items():
            params_data.append({
                'model': model_name,
                'parameter': param_name,
                'best_value': str(param_value)
            })
    
    if params_data:
        params_df = pd.DataFrame(params_data)
        params_path = os.path.join(output_dir, f"{product_name}_最佳参数.xlsx")
        params_df.to_excel(params_path, index=False)
        print(f"最佳参数已保存至: {params_path}")


# =============================================================================
# 主要预测函数 - 逐日预测版本
# =============================================================================
def daily_prediction_and_comparison_with_search(merged_df, selected_features, product_name, output_dir,
                                               use_search=True, max_time_per_model=180, search_strategy="smart"):
    """执行逐日预测并与13周移动平均法比较（集成超参数搜索）"""
    print(f"\n开始为{product_name}执行逐日预测与比较...")
    
    # 确保日期列是datetime类型并按日期排序
    merged_df['date'] = pd.to_datetime(merged_df['date'])
    merged_df = merged_df.sort_values('date').reset_index(drop=True)
    
    # 划分训练集和测试集（2024年作为测试集）
    train = merged_df[merged_df['date'] < '2024-01-01']
    test = merged_df[merged_df['date'] >= '2024-01-01']
    
    if test.empty:
        print(f"警告: {product_name}没有2024年数据，无法进行预测比较")
        return None
    
    print(f"训练集大小: {train.shape[0]}, 测试集大小: {test.shape[0]}")
    
    # 分别处理训练集和测试集，避免数据泄漏
    train_cleaned, test_cleaned = leak_free_fix_data_issues(train, test)
    print(f"清理后训练集大小: {train_cleaned.shape[0]}, 测试集大小: {test_cleaned.shape[0]}")
    
    # 加载已选择的特征（已经过滤了不可用特征）
    working_features = [f for f in selected_features if f in train_cleaned.columns]
    print(f"使用特征数量: {len(working_features)}")
    
    if len(working_features) == 0:
        print(f"警告: {product_name}没有可用特征，无法训练模型")
        return None
    
    # === 修改：使用新的训练函数 ===
    y_train = train_cleaned['demand']
    y_test = test_cleaned['demand']
    
    # 根据配置选择训练方式
    if use_search:
        print(f"\n使用超参数搜索训练模型（策略: {search_strategy}）...")
        trained_models, best_params_dict = train_models_with_hyperparameter_search(
            train_cleaned, working_features, y_train, product_name,
            max_time_per_model=max_time_per_model,
            search_strategy=search_strategy
        )
        
        # 保存最佳参数
        save_best_parameters(best_params_dict, product_name, output_dir)
        
    else:
        print(f"\n使用改进的默认参数训练模型...")
        trained_models, best_params_dict = train_models_with_improved_defaults(
            train_cleaned, working_features, y_train, product_name
        )
    
    # 创建预测器并执行预测（原逻辑保持不变）
    daily_predictors = {}
    model_performance = {}
    sequence_predictors = {}
    
    for name, model_info in trained_models.items():
        print(f"\n创建 {name} 预测器...")
        
        try:
            # 创建逐日预测器
            predictor = DailyPredictor(
                model_info['model'], 
                model_info['scaler_X'], 
                model_info['scaler_y'], 
                model_info['features'], 
                model_info['type'], 
                name
            )
            daily_predictors[name] = predictor
            
            # 执行逐日预测
            print(f"  执行 {name} 逐日预测...")
            y_pred = predictor.predict_test_period(test_cleaned, train_cleaned)
            
            # 计算性能指标
            performance = calculate_streamlined_metrics(y_test, y_pred)
            model_performance[name] = performance
            
            # 添加搜索信息
            if 'best_score' in model_info:
                performance['best_cv_score'] = model_info['best_score']
            if 'search_time' in model_info:
                performance['search_time'] = model_info['search_time']
            
            print(f"  {name} 逐日预测性能: MAE={performance['MAE']:.2f}, WMAPE={performance['WMAPE (%)']:.2f}%, R²={performance['R²']:.4f}")
            
        except Exception as e:
            print(f"  创建 {name} 预测器时出错: {str(e)}")
            continue
    
    # 使用统一的深度学习训练（启用搜索）
    unified_deep_learning_training_with_search(train_cleaned, test_cleaned, working_features, y_train, y_test, 
                              model_performance, daily_predictors, sequence_predictors, product_name,
                              enable_search=use_search)  # 根据主配置决定是否搜索
    
    # 添加基线模型：LinearRegression（保持原逻辑）
    print(f"\n训练基线模型...")
    try:
        lr_features = select_model_specific_features(
            working_features, 
            train_cleaned[working_features], 
            y_train, 
            'linear', 
            'LinearRegression'
        )
        
        X_train_lr = train_cleaned[lr_features]
        X_test_lr = test_cleaned[lr_features]
        
        X_train_lr_processed, X_test_lr_processed, y_train_lr_processed, scaler_X_lr, scaler_y_lr = get_preprocessed_data(
            X_train_lr, X_test_lr, y_train, 'linear'
        )
        
        lr_model = LinearRegression()
        lr_model.fit(X_train_lr_processed, y_train_lr_processed)
        
        lr_predictor = DailyPredictor(lr_model, scaler_X_lr, scaler_y_lr, lr_features, 'linear', 'LinearRegression')
        daily_predictors['LinearRegression'] = lr_predictor
        
        y_pred_lr = lr_predictor.predict_test_period(test_cleaned, train_cleaned)
        performance_lr = calculate_streamlined_metrics(y_test, y_pred_lr)
        model_performance['LinearRegression'] = performance_lr
        
        print(f"  LinearRegression 基线性能: MAE={performance_lr['MAE']:.2f}, WMAPE={performance_lr['WMAPE (%)']:.2f}%, R²={performance_lr['R²']:.4f}")
        
    except Exception as e:
        print(f"  训练LinearRegression基线时出错: {str(e)}")
    
    # 收集所有预测结果
    print(f"\n收集所有模型的逐日预测结果...")
    model_predictions = {}
    
    for name, predictor in daily_predictors.items():
        try:
            if hasattr(predictor, 'predict_test_period'):
                if name in sequence_predictors:
                    # 序列预测器
                    y_pred = predictor.predict_test_period(test_cleaned, train_cleaned)
                else:
                    # 普通预测器（使用A/B填充）
                    y_pred = predictor.predict_test_period(test_cleaned, train_cleaned)
                
                # 确保预测长度与测试集一致
                if len(y_pred) != len(y_test):
                    min_len = min(len(y_pred), len(y_test))
                    y_pred = y_pred[:min_len]
                
                model_predictions[name] = y_pred
            
        except Exception as e:
            print(f"  获取 {name} 预测结果时出错: {str(e)}")
            continue
    
    # 创建改进的集成预测（排除基线模型）
    if len(model_predictions) > 2:  # 确保有足够的模型进行集成
        print(f"\n创建改进的动态加权集成预测...")
        try:
            ensemble_result = create_performance_based_ensemble(
                model_predictions, model_performance, exclude_baselines=True
            )
            
            if ensemble_result[0] is not None:
                ensemble_pred, ensemble_weights = ensemble_result
                model_predictions["Improved_Ensemble"] = ensemble_pred
                
                # 计算集成模型性能
                y_test_adj = y_test.iloc[:len(ensemble_pred)]
                if len(y_test_adj) == len(ensemble_pred):
                    performance = calculate_streamlined_metrics(y_test_adj, ensemble_pred)
                    model_performance["Improved_Ensemble"] = performance
                    print(f"Improved_Ensemble 预测性能: MAE={performance['MAE']:.2f}, WMAPE={performance['WMAPE (%)']:.2f}%, R²={performance['R²']:.4f}")
                
        except Exception as e:
            print(f"  创建改进集成模型时出错: {str(e)}")
    
    # 13周移动平均法预测（基准方法）
    print(f"\n执行13周移动平均法预测...")
    weekly_demand = merged_df.resample('W-SUN', on='date')['demand'].sum().reset_index()
    weekly_demand.columns = ['week_start', 'actual_weekly_demand']
    weekly_demand['ma_13'] = weekly_demand['actual_weekly_demand'].rolling(window=13, min_periods=1).mean()
    weekly_demand['ma_forecast'] = weekly_demand['ma_13'].shift(1)
    
    weekly_demand_2024 = weekly_demand[weekly_demand['week_start'] >= '2024-01-01']
    
    # 将逐日预测结果按周汇总
    test_results = test_cleaned[['date', 'demand']].copy()
    
    # 处理不同长度的预测结果
    for name, pred in model_predictions.items():
        pred_len = len(pred)
        test_len = len(test_results)
        
        if pred_len == test_len:
            test_results[f'{name}_prediction'] = pred
        elif pred_len < test_len:
            # 预测结果较短，用最后一个值填充
            pred_extended = np.concatenate([pred, np.full(test_len - pred_len, pred[-1] if pred_len > 0 else 0)])
            test_results[f'{name}_prediction'] = pred_extended
        else:
            # 预测结果较长，截断
            test_results[f'{name}_prediction'] = pred[:test_len]
    
    # 按周汇总
    agg_dict = {'demand': 'sum'}
    for name in model_predictions.keys():
        pred_col = f'{name}_prediction'
        if pred_col in test_results.columns:
            agg_dict[pred_col] = 'sum'
    
    weekly_model_results = test_results.resample('W-SUN', on='date').agg(agg_dict).reset_index()
    
    # 重命名列
    new_columns = ['week_start', 'actual_weekly_demand']
    for name in model_predictions.keys():
        pred_col = f'{name}_prediction'
        if pred_col in weekly_model_results.columns:
            new_columns.append(f'{name}_forecast')
    
    weekly_model_results.columns = new_columns
    
    # 合并结果
    comparison_df = pd.merge(
        weekly_model_results, 
        weekly_demand_2024[['week_start', 'ma_forecast']], 
        on='week_start', 
        how='left'
    )
    
    # 计算每周性能指标
    weekly_metrics = {}
    
    for col in comparison_df.columns:
        if col.endswith('_forecast'):
            if col == 'ma_forecast':
                model_name = '13周移动平均'
            else:
                model_name = col.replace('_forecast', '')
            
            if not comparison_df[col].isna().all():
                try:
                    metrics = calculate_streamlined_metrics(
                        comparison_df['actual_weekly_demand'], 
                        comparison_df[col]
                    )
                    weekly_metrics[model_name] = metrics
                    
                except Exception as e:
                    print(f"  计算 {model_name} 周度指标时出错: {str(e)}")
    
    # 计算模型综合排名
    ranking_result = calculate_model_ranking(weekly_metrics, exclude_baselines=True)
    
    # 输出结果
    print(f"\n{product_name}逐日预测转周度汇总性能比较:")
    print("="*60)
    
    # 定义要显示的指标
    metrics_to_show = ['MAE', 'WMAPE (%)', 'MAPE (%)', 'RMSE', 'R²']
    
    for metric_name in metrics_to_show:
        print(f"\n{metric_name}排名:")
        metric_scores = {}
        for model, metrics in weekly_metrics.items():
            if metric_name in metrics:
                metric_scores[model] = metrics[metric_name]
        
        # 排序（R²按降序，其他按升序）
        reverse_sort = (metric_name == 'R²')
        sorted_scores = sorted(metric_scores.items(), key=lambda x: x[1], reverse=reverse_sort)
        
        for i, (model, score) in enumerate(sorted_scores, 1):
            if metric_name == 'R²':
                print(f"  {i}. {model}: {score:.4f}")
            elif '%' in metric_name:
                print(f"  {i}. {model}: {score:.2f}%")
            else:
                print(f"  {i}. {model}: {score:.0f}")
    
    # 可视化比较
    plt.figure(figsize=(20, 12))

    # 实际需求 - 使用粗实线和特殊标记
    plt.plot(comparison_df['week_start'], comparison_df['actual_weekly_demand'], 
            'o-', label='实际需求', linewidth=3, markersize=8, color='black')

    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
    color_idx = 0

    # 优先显示非基线模型 - 使用虚线
    baseline_models = ['LinearRegression', '13周移动平均']

    for model in model_predictions.keys():
        if model not in baseline_models:
            forecast_col = f'{model}_forecast'
            if forecast_col in comparison_df.columns:
                plt.plot(comparison_df['week_start'], comparison_df[forecast_col], 
                        '--', label=f'{model}预测', linewidth=2, 
                        color=colors[color_idx % len(colors)])
                color_idx += 1

    # 最后显示基线模型 - 使用点划线和较浅的颜色
    baseline_linestyles = ['-.', '-.']  # 点划线
    baseline_alphas = [0.6, 0.6]  # 半透明
    baseline_linewidths = [2.5, 2.5]  # 稍粗一点

    for idx, model in enumerate(baseline_models):
        if model == '13周移动平均':
            forecast_col = 'ma_forecast'
        else:
            forecast_col = f'{model}_forecast'
        
        if forecast_col in comparison_df.columns and not comparison_df[forecast_col].isna().all():
            plt.plot(comparison_df['week_start'], comparison_df[forecast_col], 
                    baseline_linestyles[idx], 
                    label=f'{model}(基线)', 
                    linewidth=baseline_linewidths[idx], 
                    alpha=baseline_alphas[idx],
                    color=colors[color_idx % len(colors)])
            color_idx += 1

    plt.title(f'{product_name}逐日预测汇总每周需求比较 (2024年)', fontsize=16)
    plt.xlabel('周起始日期', fontsize=14)
    plt.ylabel('每周需求', fontsize=14)
    plt.legend(fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    plot_path = os.path.join(output_dir, f"{product_name}_2024年逐日预测汇总比较.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"\n预测比较图表保存至: {plot_path}")
    
    # 保存详细结果
    result_path = os.path.join(output_dir, f"{product_name}_2024年逐日预测汇总结果.xlsx")
    comparison_df.to_excel(result_path, index=False)
    print(f"详细结果保存至: {result_path}")
    
    # 保存逐日预测结果
    daily_result_path = os.path.join(output_dir, f"{product_name}_2024年逐日预测详细结果.xlsx")
    test_results.to_excel(daily_result_path, index=False)
    print(f"逐日预测详细结果保存至: {daily_result_path}")
    
    # ===== 15日聚合评估 =====
    bi15d_model_results = test_results.resample('15D', on='date').agg(agg_dict).reset_index()
    bi15d_model_results = bi15d_model_results.rename(columns={'date': 'period_start', 'demand': 'actual_period_demand'})
    
    # 计算15日聚合指标（无基线）
    bi15d_metrics = {}
    for col in bi15d_model_results.columns:
        if col.endswith('_prediction'):
            model_name = col.replace('_prediction', '')
            try:
                metrics = calculate_streamlined_metrics(bi15d_model_results['actual_period_demand'], bi15d_model_results[col])
                bi15d_metrics[model_name] = metrics
            except Exception as e:
                print(f"  计算 {model_name} 15日指标时出错: {str(e)}")
    
    # 保存15日汇总
    result_15d_path = os.path.join(output_dir, f"{product_name}_2024年逐日预测汇总结果_15D.xlsx")
    bi15d_model_results.to_excel(result_15d_path, index=False)
    print(f"详细结果保存至(15D): {result_15d_path}")
    
    # ===== 14日聚合评估 =====
    bi14d_model_results = test_results.resample('14D', on='date').agg(agg_dict).reset_index()
    bi14d_model_results = bi14d_model_results.rename(columns={'date': 'period_start', 'demand': 'actual_period_demand'})
    
    bi14d_metrics = {}
    for col in bi14d_model_results.columns:
        if col.endswith('_prediction'):
            model_name = col.replace('_prediction', '')
            try:
                metrics = calculate_streamlined_metrics(bi14d_model_results['actual_period_demand'], bi14d_model_results[col])
                bi14d_metrics[model_name] = metrics
            except Exception as e:
                print(f"  计算 {model_name} 14日指标时出错: {str(e)}")
    
    result_14d_path = os.path.join(output_dir, f"{product_name}_2024年逐日预测汇总结果_14D.xlsx")
    bi14d_model_results.to_excel(result_14d_path, index=False)
    print(f"详细结果保存至(14D): {result_14d_path}")
    
    # ===== 30日聚合评估 =====
    bi30d_model_results = test_results.resample('30D', on='date').agg(agg_dict).reset_index()
    bi30d_model_results = bi30d_model_results.rename(columns={'date': 'period_start', 'demand': 'actual_period_demand'})
    
    bi30d_metrics = {}
    for col in bi30d_model_results.columns:
        if col.endswith('_prediction'):
            model_name = col.replace('_prediction', '')
            try:
                metrics = calculate_streamlined_metrics(bi30d_model_results['actual_period_demand'], bi30d_model_results[col])
                bi30d_metrics[model_name] = metrics
            except Exception as e:
                print(f"  计算 {model_name} 30日指标时出错: {str(e)}")
    
    result_30d_path = os.path.join(output_dir, f"{product_name}_2024年逐日预测汇总结果_30D.xlsx")
    bi30d_model_results.to_excel(result_30d_path, index=False)
    print(f"详细结果保存至(30D): {result_30d_path}")
    
    # 保存模型性能
    if model_performance:
        performance_df = pd.DataFrame(model_performance).T.reset_index().rename(columns={'index': 'model'})
        performance_path = os.path.join(output_dir, f"{product_name}_逐日预测模型性能对比.xlsx")
        performance_df.to_excel(performance_path, index=False)
        print(f"模型性能对比保存至: {performance_path}")
    
    return {
        'weekly_metrics': weekly_metrics,
        'comparison_df': comparison_df,
        'daily_results': test_results,
        'bi15d_results': bi15d_model_results,
        'bi15d_metrics': bi15d_metrics,
        'model_performance': model_performance,
        'daily_predictors': daily_predictors,
        'ranking': ranking_result,
        'best_params': best_params_dict,  # 新增：最佳参数
        'bi14d_results': bi14d_model_results,
        'bi14d_metrics': bi14d_metrics,
        'bi30d_results': bi30d_model_results,
        'bi30d_metrics': bi30d_metrics
    }

# =============================================================================
# 主程序
# =============================================================================
def main():
    """逐日预测版主程序"""
    # === 新增：超参数搜索配置 ===
    USE_HYPERPARAMETER_SEARCH = True  # 改为True启用搜索，False使用默认参数
    MAX_TIME_PER_MODEL = 180  # 每个模型最大搜索时间（秒）
    SEARCH_STRATEGY = "smart"  # "smart", "grid", "random"
    
    print(f"超参数搜索: {'启用' if USE_HYPERPARAMETER_SEARCH else '禁用'}")
    if USE_HYPERPARAMETER_SEARCH:
        print(f"搜索策略: {SEARCH_STRATEGY}")
        print(f"每模型最大时间: {MAX_TIME_PER_MODEL}秒")
    
    base_dir = r"/Users/bytedance/Desktop/特征工程补充/结果"
    products = ['电解镍', '高碳铬铁', '铝锭']
    
    all_results = {}
    
    for product in products:
        print(f"\n{'='*80}")
        print(f"开始处理 {product} 的2024年逐日预测比较")
        print(f"{'='*80}")
        
        product_output_dir = os.path.join(base_dir, f"{product}预测结果")
        os.makedirs(product_output_dir, exist_ok=True)
        print(f"输出目录: {product_output_dir}")
        
        # 1. 加载选定的特征
        selected_features = load_selected_features(product, base_dir)
        print(f"已加载{product}的选定特征: {len(selected_features)}个")
        
        # 2. 读取特征工程数据
        feature_file = os.path.join(product_output_dir, f"{product}_特征工程数据.xlsx")
        if not os.path.exists(feature_file):
            print(f"警告: 找不到 {product} 的特征工程数据文件: {feature_file}")
            continue
        
        try:
            merged_df = pd.read_excel(feature_file)
            
            if merged_df.empty:
                print(f"警告: {product} 的特征工程数据为空，跳过处理")
                continue
            
            merged_df['date'] = pd.to_datetime(merged_df['date'], errors='coerce')
            print(f"已加载 {product} 的特征工程数据，形状: {merged_df.shape}")
            
            # 3. 逐日预测比较 - 传入搜索配置
            comparison_results = daily_prediction_and_comparison_with_search(
                merged_df, selected_features, product, product_output_dir,
                use_search=USE_HYPERPARAMETER_SEARCH,
                max_time_per_model=MAX_TIME_PER_MODEL,
                search_strategy=SEARCH_STRATEGY
            )
            
            if comparison_results:
                all_results[product] = comparison_results
            
            print(f"\n{product} 的逐日预测比较完成!")
            
        except Exception as e:
            print(f"处理 {product} 时出错: {str(e)}")
            import traceback
            traceback.print_exc()
            continue
    
    # 4. 生成汇总报告
    print(f"\n{'='*80}")
    print(f"生成逐日预测汇总报告")
    print(f"{'='*80}")
    
    generate_summary_report(all_results, base_dir)
    
    print("\n所有逐日预测处理完成!")

def generate_summary_report(all_results, base_dir):
    """生成逐日预测汇总报告"""
    print("\n生成逐日预测汇总报告...")
    
    if not all_results:
        print("没有结果可以汇总")
        return
    
    # 收集所有指标数据
    summary_data = []
    
    for product, results in all_results.items():
        weekly_metrics = results.get('weekly_metrics', {})
        
        for model_name, metrics in weekly_metrics.items():
            summary_data.append({
                '产品': product,
                '模型': model_name,
                'MAE': metrics.get('MAE', np.nan),
                'RMSE': metrics.get('RMSE', np.nan),
                'R²': metrics.get('R²', np.nan),
                'MAPE (%)': metrics.get('MAPE (%)', np.nan),
                'WMAPE (%)': metrics.get('WMAPE (%)', np.nan)
            })
    
    if summary_data:
        summary_df = pd.DataFrame(summary_data)
        
        # 保存汇总数据
        summary_path = os.path.join(base_dir, "逐日预测汇总结果.xlsx")
        summary_df.to_excel(summary_path, index=False)
        print(f"  逐日预测汇总结果已保存至: {summary_path}")
        
        # 生成各指标的最佳模型汇总
        metrics_to_analyze = ['MAE', 'RMSE', 'MAPE (%)', 'WMAPE (%)', 'R²']
        best_models_summary = []
        
        for metric in metrics_to_analyze:
            print(f"\n基于{metric}的各产品最佳模型:")
            
            for product in summary_df['产品'].unique():
                product_data = summary_df[summary_df['产品'] == product]
                
                if metric == 'R²':
                    # R²越大越好
                    best_idx = product_data[metric].idxmax()
                    best_value = product_data[metric].max()
                else:
                    # 其他指标越小越好
                    best_idx = product_data[metric].idxmin()
                    best_value = product_data[metric].min()
                
                if not pd.isna(best_value):
                    best_model = product_data.loc[best_idx, '模型']
                    
                    if metric == 'R²':
                        print(f"- {product}: {best_model} ({metric}: {best_value:.4f})")
                        best_models_summary.append({
                            '产品': product,
                            '指标': metric,
                            '最佳模型': best_model,
                            '最佳值': f"{best_value:.4f}"
                        })
                    elif '%' in metric:
                        print(f"- {product}: {best_model} ({metric}: {best_value:.2f}%)")
                        best_models_summary.append({
                            '产品': product,
                            '指标': metric,
                            '最佳模型': best_model,
                            '最佳值': f"{best_value:.2f}%"
                        })
                    else:
                        print(f"- {product}: {best_model} ({metric}: {best_value:.0f})")
                        best_models_summary.append({
                            '产品': product,
                            '指标': metric,
                            '最佳模型': best_model,
                            '最佳值': f"{best_value:.0f}"
                        })
        
        if best_models_summary:
            best_models_df = pd.DataFrame(best_models_summary)
            best_models_path = os.path.join(base_dir, "逐日预测最佳模型汇总.xlsx")
            best_models_df.to_excel(best_models_path, index=False)
            print(f"  逐日预测最佳模型汇总已保存至: {best_models_path}")
        
        # 生成热力图 - 每个品种一个图，包含所有指标，每个指标独立着色
        try:
            # 检查实际可用的指标
            available_metrics = [col for col in summary_df.columns if col not in ['产品', '模型']]
            print(f"  可用指标: {available_metrics}")
            
            # 选择要显示的指标（确保都存在于数据中）
            desired_metrics = ['MAE', 'RMSE', 'MAPE (%)', 'WMAPE (%)', 'R²']
            metrics_for_heatmap = [m for m in desired_metrics if m in available_metrics]
            
            print(f"  将显示的指标: {metrics_for_heatmap}")
            
            if not metrics_for_heatmap:
                print("  警告: 没有可用的指标数据生成热力图")
                return
            
            # 获取所有产品
            products = summary_df['产品'].unique()
            n_products = len(products)
            
            # 动态调整子图布局
            if n_products == 1:
                nrows, ncols = 1, 1
            elif n_products == 2:
                nrows, ncols = 1, 2
            else:
                nrows, ncols = 2, 2
            
            fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*12, nrows*10))
            if n_products == 1:
                axes = [axes]
            elif nrows == 1:
                axes = axes if hasattr(axes, '__len__') else [axes]
            else:
                axes = axes.flatten()
            
            for i, product in enumerate(products):
                if i < len(axes):
                    # 获取该产品的数据
                    product_data = summary_df[summary_df['产品'] == product]
                    
                    if product_data.empty:
                        axes[i].text(0.5, 0.5, f'{product}\n无数据', 
                                   ha='center', va='center', transform=axes[i].transAxes)
                        axes[i].set_title(f'{product}模型性能对比', fontsize=14)
                        continue
                    
                    # 准备热力图数据 - 模型为行，指标为列
                    heatmap_data = product_data.set_index('模型')[metrics_for_heatmap]

                    # 检查数据是否为空
                    if heatmap_data.empty or heatmap_data.isna().all().all():
                        axes[i].text(0.5, 0.5, f'{product}\n无有效数据', 
                                ha='center', va='center', transform=axes[i].transAxes)
                        axes[i].set_title(f'{product}模型性能对比', fontsize=14)
                        continue

                    # ===== 新增：自定义排序 =====
                    # 定义排序优先级：13周移动平均 > LinearRegression > 其他模型 > Improved_Ensemble
                    def custom_sort_key(model_name):
                        if model_name == '13周移动平均':
                            return (0, model_name)
                        elif model_name == 'LinearRegression':
                            return (1, model_name)
                        elif model_name == 'Improved_Ensemble':
                            return (3, model_name)
                        else:
                            return (2, model_name)

                    # 应用排序
                    sorted_index = sorted(heatmap_data.index, key=custom_sort_key)
                    heatmap_data = heatmap_data.loc[sorted_index]
                    # ===== 排序结束 =====

                    # 创建标准化后的数据用于着色，但显示原始值
                    normalized_data = heatmap_data.copy()
                    annotations = heatmap_data.copy()

                    
                    # 对每个指标进行独立标准化和格式化
                    for metric in metrics_for_heatmap:
                        if metric in heatmap_data.columns:
                            metric_values = heatmap_data[metric].dropna()
                            if len(metric_values) > 0:
                                # 计算该指标的最小值和最大值
                                min_val = metric_values.min()
                                max_val = metric_values.max()
                                
                                if max_val != min_val:
                                    if metric == 'R²':
                                        # R²: 越大越好，直接标准化到0-1
                                        normalized_data[metric] = (heatmap_data[metric] - min_val) / (max_val - min_val)
                                    else:
                                        # 其他指标: 越小越好，反向标准化
                                        normalized_data[metric] = 1 - (heatmap_data[metric] - min_val) / (max_val - min_val)
                                else:
                                    # 如果最大值等于最小值，设为中间值
                                    normalized_data[metric] = 0.5
                                
                                # 格式化注释文本
                                if metric == 'R²':
                                    annotations[metric] = heatmap_data[metric].apply(lambda x: f'{x:.3f}' if pd.notna(x) else 'N/A')
                                elif '%' in metric:
                                    annotations[metric] = heatmap_data[metric].apply(lambda x: f'{x:.1f}%' if pd.notna(x) else 'N/A')
                                else:
                                    annotations[metric] = heatmap_data[metric].apply(lambda x: f'{x:.0f}' if pd.notna(x) else 'N/A')
                    
                    # 绘制热力图
                    sns.heatmap(normalized_data, 
                               annot=annotations, 
                               fmt='', 
                               cmap='RdYlGn',  # 使用统一色彩，但每个指标独立标准化
                               cbar_kws={'label': '相对性能（绿色=较好）'}, 
                               linewidths=0.5, 
                               ax=axes[i],
                               vmin=0, vmax=1)  # 固定色彩范围
                    
                    axes[i].set_title(f'{product} - 各模型性能对比', fontsize=14, pad=15)
                    axes[i].set_xlabel('评估指标', fontsize=12)
                    axes[i].set_ylabel('模型', fontsize=12)
                    axes[i].tick_params(axis='x', rotation=45)
                    axes[i].tick_params(axis='y', rotation=0)
            
            # 隐藏多余的子图
            for j in range(n_products, len(axes)):
                axes[j].set_visible(False)
            
            plt.suptitle('各产品模型性能对比热力图', fontsize=16, y=0.95)
            plt.tight_layout(rect=[0, 0, 1, 0.92])
            
            heatmap_path = os.path.join(base_dir, "逐日预测各产品独立指标热力图.png")
            plt.savefig(heatmap_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"  逐日预测各产品独立指标热力图已保存至: {heatmap_path}")
            
        except Exception as e:
            print(f"  生成热力图时出错: {str(e)}")
            import traceback
            traceback.print_exc()
        
        # 改进的模型一致性分析
        try:
            print(f"\n逐日预测模型一致性分析 (基于WMAPE40%+MAE30%+R²30%):")
            consistency_analysis = {}
            
            for product in summary_df['产品'].unique():
                product_data = summary_df[summary_df['产品'] == product]
                
                # 排除基线模型进行分析
                baseline_models = ['LinearRegression', '13周移动平均']
                non_baseline_data = product_data[~product_data['模型'].isin(baseline_models)]
                
                if len(non_baseline_data) == 0:
                    continue
                
                best_models = {}
                
                # 核心指标
                core_metrics = ['MAE', 'RMSE', 'MAPE (%)', 'WMAPE (%)']
                
                # 处理需要最小化的指标
                for metric in core_metrics:
                    if metric in non_baseline_data.columns:
                        best_idx = non_baseline_data[metric].idxmin()
                        if not pd.isna(non_baseline_data.loc[best_idx, metric]):
                            best_models[metric] = non_baseline_data.loc[best_idx, '模型']
                
                # R²单独处理（需要最大化）
                if 'R²' in non_baseline_data.columns:
                    r2_best_idx = non_baseline_data['R²'].idxmax()
                    if not pd.isna(non_baseline_data.loc[r2_best_idx, 'R²']):
                        best_models['R²'] = non_baseline_data.loc[r2_best_idx, '模型']
                
                if best_models:
                    models_list = list(best_models.values())
                    most_common = max(set(models_list), key=models_list.count)
                    consistency_count = models_list.count(most_common)
                    
                    # 计算加权一致性分数
                    weights = {'WMAPE (%)': 0.4, 'MAE': 0.3, 'R²': 0.3}
                    weighted_score = 0
                    total_weight = 0
                    
                    for metric, model in best_models.items():
                        if metric in weights:
                            weight = weights[metric]
                            if model == most_common:
                                weighted_score += weight
                            total_weight += weight
                        else:
                            # 其他指标权重较小
                            weight = 0.05
                            if model == most_common:
                                weighted_score += weight
                            total_weight += weight
                    
                    weighted_consistency = weighted_score / total_weight if total_weight > 0 else 0
                    
                    consistency_analysis[product] = {
                        **{f'{metric}最佳': model for metric, model in best_models.items()},
                        '推荐模型': most_common,
                        '一致性得分': f"{consistency_count}/{len(best_models)}",
                        '加权一致性': f"{weighted_consistency:.2f}"
                    }
                    
                    # 构建显示字符串，突出核心指标
                    core_metrics_str = []
                    other_metrics_str = []
                    
                    for metric, model in best_models.items():
                        if metric in ['WMAPE (%)', 'MAE', 'R²']:
                            core_metrics_str.append(f"{metric}→{model}")
                        else:
                            other_metrics_str.append(f"{metric}→{model}")
                    
                    display_str = " | ".join(core_metrics_str)
                    if other_metrics_str:
                        display_str += f" | 其他: {', '.join(other_metrics_str)}"
                    
                    print(f"- {product}: {display_str}")
                    print(f"  → 推荐: {most_common} (一致性: {consistency_count}/{len(best_models)}, 加权: {weighted_consistency:.2f})")
            
            if consistency_analysis:
                consistency_df = pd.DataFrame(consistency_analysis).T.reset_index().rename(columns={'index': '产品'})
                consistency_path = os.path.join(base_dir, "逐日预测模型一致性分析.xlsx")
                consistency_df.to_excel(consistency_path, index=False)
                print(f"  逐日预测模型一致性分析已保存至: {consistency_path}")
        
        except Exception as e:
            print(f"  模型一致性分析时出错: {str(e)}")
            import traceback
            traceback.print_exc()
        
        # 生成最终推荐汇总
        try:
            print(f"\n最终模型推荐汇总:")
            recommendations = []
            
            for product, results in all_results.items():
                ranking = results.get('ranking', [])
                if ranking:
                    # 获取排名前3的模型
                    top_3 = ranking[:3]
                    best_model = top_3[0][0]
                    best_score = top_3[0][1]
                    
                    print(f"- {product}: 推荐 {best_model} (综合得分: {best_score:.2f})")
                    print(f"  前三名: {', '.join([f'{model}({score:.1f}分)' for model, score in top_3])}")
                    
                    recommendations.append({
                        '产品': product,
                        '推荐模型': best_model,
                        '综合得分': f"{best_score:.2f}",
                        '前三名': ', '.join([f'{model}({score:.1f})' for model, score in top_3])
                    })
            
            if recommendations:
                recommendations_df = pd.DataFrame(recommendations)
                recommendations_path = os.path.join(base_dir, "最终模型推荐汇总.xlsx")
                recommendations_df.to_excel(recommendations_path, index=False)
                print(f"  最终模型推荐汇总已保存至: {recommendations_path}")
                
        except Exception as e:
            print(f"  生成最终推荐时出错: {str(e)}")
    
    print(f"\n逐日预测汇总报告已保存到: {base_dir}")

if __name__ == "__main__":
    main()                

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


超参数搜索: 启用
搜索策略: smart
每模型最大时间: 180秒

开始处理 电解镍 的2024年逐日预测比较
输出目录: /Users/bytedance/Desktop/特征工程补充/结果/电解镍预测结果
已加载电解镍的选定特征: 40个
已加载 电解镍 的特征工程数据，形状: (1521, 248)

开始为电解镍执行逐日预测与比较...
训练集大小: 1461, 测试集大小: 60
执行无数据泄漏的数据清理...
清理后训练集大小: 1461, 测试集大小: 60
使用特征数量: 40

使用超参数搜索训练模型（策略: smart）...
开始超参数搜索训练（策略: smart）...

训练 RandomForest 模型...
  RandomForest 使用 32 个特征
    数据量: 1461
    使用随机搜索（数据量 >= 200）
    搜索完成，用时: 21.6秒
    最佳参数: {'max_depth': 8, 'min_samples_leaf': 4, 'min_samples_split': 7, 'n_estimators': 441}
    最佳得分: -11966.0613
  RandomForest 搜索完成，用时: 21.6秒

训练 XGBoost 模型...
  XGBoost 使用 32 个特征
    数据量: 1461
    使用随机搜索（数据量 >= 200）
    搜索完成，用时: 187.9秒
    最佳参数: {'colsample_bytree': np.float64(0.8540922615763339), 'gamma': np.float64(0.2252496259847715), 'learning_rate': np.float64(0.00626017131018732), 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 313, 'reg_alpha': np.float64(0.16167946962329224), 'reg_lambda': np.float64(1.2615344229334267), 'subsample': np.float64(0.6793016342019151)}